# VCF Statistics Analysis Workflow

**Purpose**: Comprehensive VCF statistics analysis with unified CxDy tiering system for DNA/RNA variant calling pipelines.

**Key Features:**
- 🔄 **Dual Workflow Support**: Standard + Realignment (RNA-specific)
- 📊 **CxDy Tiering System**: C1-C7 (Caller Support) × D0-D1 (Database Evidence)
- 🎯 **Stage-Aware Classification**: Somatic, Germline, Reference, Artifact, RNAedit, NoConsensus
- 📈 **Comprehensive Visualizations**: Pipeline progression, caller comparisons, tier distributions
- ✅ **BAM Validation**: Variant support verification across tumor/normal samples
- 🧬 **Caller Comparison**: 3-way Venn diagrams (Strelka, DeepSomatic, Mutect2)

**Required Configuration**: Edit the next cell with your sample-specific paths before running the analysis.

In [1]:
# ==================================================================================
# CONFIGURATION - EDIT THIS CELL FOR YOUR SAMPLE
# ==================================================================================

from pathlib import Path

# Sample Configuration
SAMPLE_ID = "P2374372"  # Change this to your sample ID

# Base directory containing VCF analysis results
# Expected structure: {BASE_DIR}/variant_calling/, {BASE_DIR}/consensus/, etc.
BASE_DIR = Path(f"/t9k/mnt/hdd/work/Vax/sequencing/aim_exp/aim_11/output/{SAMPLE_ID}")

# Output directory for statistics and visualizations
OUTPUT_DIR = Path(f"{SAMPLE_ID}_statistics_output")

# Reference genome FASTA (required for IGV-reports)
REF_FASTA = Path(
    "/t9k/mnt/joey/bio_db/references/Homo_sapiens/GATK/GRCh38/Sequence/WholeGenomeFasta/Homo_sapiens_assembly38.fasta"
)

# Sample naming conventions (usually no need to change)
SAMPLE_SUFFIX_MAP = {
    "DT": "DNA_TUMOR",
    "DN": "DNA_NORMAL",
    "RT": "RNA_TUMOR"
}

# ==================================================================================
# VALIDATION - DO NOT EDIT
# ==================================================================================

# Validate paths
if not BASE_DIR.exists():
    raise FileNotFoundError(f"❌ Base directory not found: {BASE_DIR}")

if not REF_FASTA.exists():
    print(f"⚠️  Warning: Reference FASTA not found: {REF_FASTA}")
    print("   IGV-reports will be skipped")

# Create output directory
OUTPUT_DIR.mkdir(exist_ok=True)

print("✅ Configuration validated")
print(f"   Sample ID: {SAMPLE_ID}")
print(f"   Base directory: {BASE_DIR}")
print(f"   Output directory: {OUTPUT_DIR}")
print(f"   Reference FASTA: {'Found' if REF_FASTA.exists() else 'Missing'}")

✅ Configuration validated
   Sample ID: P2374372
   Base directory: /t9k/mnt/hdd/work/Vax/sequencing/aim_exp/aim_11/output/P2374372
   Output directory: P2374372_statistics_output
   Reference FASTA: Found


## Setup and Imports

This section imports the unified VCF statistics analysis package and configures the Python environment.

**Module Organization:**
- `bin/vcf_stats/` - Main analysis package (workflow management, file discovery, statistics, visualization)
- `bin/common/` - Shared configuration (vcf_config.py, tier_config.py)

**Key Components:**
- **Workflow Management**: WorkflowManager, WorkflowType, WorkflowComparator
- **File Discovery**: VCFFileDiscovery (stage-aware, workflow-aware)
- **Statistics**: VCFStatisticsExtractor, StatisticsAggregator, process_all_vcfs
- **Visualization**: VCFVisualizer (pipeline plots, tier distributions)
- **Validation**: BAMValidator, RealignmentBAMValidator
- **Caller Comparison**: CallerComparator (3-way Venn diagrams)
- **Tiering**: tier_rescue_variants, TierVisualizer (CxDy system)

In [2]:
# Import core modules
import sys
from pathlib import Path
import pandas as pd

# Add bin/ directory to path for vcf_stats and common modules
bin_path = Path.cwd().parent / "bin"
if str(bin_path) not in sys.path:
    sys.path.insert(0, str(bin_path))

# Add bin/common for shared config modules
bin_common_path = bin_path / "common"
if str(bin_common_path) not in sys.path:
    sys.path.insert(0, str(bin_common_path))

# Import all required modules
from vcf_stats import (
    # Workflow management
    WorkflowManager,
    WorkflowType,
    WorkflowComparator,
    # File discovery
    VCFFileDiscovery,
    # Statistics
    VCFStatisticsExtractor,
    StatisticsAggregator,
    process_all_vcfs,
    # Visualization
    VCFVisualizer,
    # BAM Validation
    BAMValidator,
    RealignmentBAMValidator,
    # Rescue analysis
    analyze_rescue_vcf,
    export_rescue_analysis,
    # Tiering
    tier_rescue_variants,
    TierVisualizer,
    create_tier_visualization_report,
    # IGV Reports
    check_igv_reports_available,
    get_alignment_index_path,
    organize_by_category_tier,
    # Caller comparison
    CallerComparator,
    extract_classified_variant_sets,
    compute_venn_overlaps,
    compute_3way_venn_plotly,
    # Constants
    TOOLS,
    MODALITIES,
    CATEGORY_ORDER,
    VCF_STAGE_ORDER,
    CATEGORY_COLORS,
)

# Import tier configuration for CxDy tiering system
from tier_config import TIER_ORDER, TIER_COLORS, get_tier_display_name

print("✅ VCF statistics modules imported successfully")
print(f"   - Workflow types: {[wt.value for wt in WorkflowType]}")
print(f"   - Processing stages: {len(VCF_STAGE_ORDER)} stages")
print(f"   - Categories: {', '.join(CATEGORY_ORDER)}")
print(f"   - Tiering system: CxDy format ({len(TIER_ORDER)} tiers: C1-C7 × D0-D1)")

✓ Using shared vcf_config module from bin/common/
✓ Variant classification functions loaded from shared modules (stage-aware)
✓ Workflow Comparison module loaded successfully
✓ Enhanced Statistics Aggregator imported successfully
✓ VCF statistics utility functions loaded
✓ VCF Statistics Extractor with stage-aware classification loaded
✓ VCF Visualizer module loaded successfully
✓ VCF statistics core module initialized
✅ VCF statistics modules imported successfully
   - Workflow types: ['standard', 'realignment']
   - Processing stages: 6 stages
   - Categories: Somatic, Germline, Reference, Artifact, RNAedit, NoConsensus
   - Tiering system: CxDy format (14 tiers: C1-C7 × D0-D1)


In [3]:
# Helper functions for sample naming normalization
def extract_sample_suffix(sample_name: str) -> str:
    """Extract the sample suffix (DT, DN, or RT) from a sample name."""
    for suffix in ["DT", "DN", "RT"]:
        if sample_name.endswith(suffix):
            return suffix
    return None

def map_suffix_to_modality(suffix: str) -> str:
    """Map sample suffix to modality name."""
    return SAMPLE_SUFFIX_MAP.get(suffix, suffix)

def normalize_bam_key(sample_name: str) -> str:
    """Normalize BAM file sample names to modality names."""
    suffix = extract_sample_suffix(sample_name)
    if suffix:
        return map_suffix_to_modality(suffix)
    return sample_name

def normalize_bam_files(bam_dict: dict) -> dict:
    """Normalize all BAM file keys to modality names."""
    return {normalize_bam_key(key): path for key, path in bam_dict.items()}

print("✓ Helper functions defined")

✓ Helper functions defined


## 2. Workflow Detection and VCF Discovery

Use the unified `WorkflowManager` to automatically detect available workflows and `VCFFileDiscovery` with workflow integration for comprehensive file discovery.

### Annotation Stage Detection

**Note:** The `cosmic_gnomad` and `rna_editing` annotation stages don't have their own directories. Their VCF files are stored in `rescue/` subdirectories with specific patterns:
- `cosmic_gnomad`: `*.rescue.cosmic_gnomad_annotated.final.vcf.gz`
- `rna_editing`: `*.rescue.rna_annotated.vcf.gz`

The workflow manager now properly detects these stages by checking for files in the rescue directory.

In [4]:
# Initialize workflow manager and detect workflows
print("=" * 80)
print("WORKFLOW DETECTION")
print("=" * 80)

workflow_manager = WorkflowManager(BASE_DIR)
detected_workflows = workflow_manager.detect_workflows()

print(f"\nDetected workflows: {[wf.value for wf in detected_workflows]}")

# Get configurations for all detected workflows
workflow_configs = workflow_manager.get_all_configs()

for wf_type, config in workflow_configs.items():
    print(f"\n{wf_type.value.upper()} Workflow:")
    print(f"  Base path: {config.base_path}")
    print(f"  Stages: {', '.join(config.stages)}")
    available_stages = workflow_manager.get_available_stages(wf_type)
    print(f"  Available stages: {', '.join(available_stages)}")
    
    # Highlight annotation stages
    annotation_stages = [s for s in available_stages if s in ['cosmic_gnomad', 'rna_editing']]
    if annotation_stages:
        print(f"  ✓ Annotation stages: {', '.join(annotation_stages)}")

WORKFLOW DETECTION

Detected workflows: ['standard', 'realignment']

STANDARD Workflow:
  Base path: /t9k/mnt/hdd/work/Vax/sequencing/aim_exp/aim_11/output/P2374372
  Stages: normalized, consensus, rescue, cosmic_gnomad, rna_editing, filtered_rescue
  Available stages: normalized, consensus, rescue, cosmic_gnomad, rna_editing, filtered_rescue
  ✓ Annotation stages: cosmic_gnomad, rna_editing

REALIGNMENT Workflow:
  Base path: /t9k/mnt/hdd/work/Vax/sequencing/aim_exp/aim_11/output/P2374372/vcf_realignment
  Stages: normalized, consensus, rescue, cosmic_gnomad, rna_editing, filtered_rescue
  Available stages: normalized, consensus, rescue, cosmic_gnomad, rna_editing, filtered_rescue
  ✓ Annotation stages: cosmic_gnomad, rna_editing


In [5]:
# Discover VCF files for ALL workflows in a single call
print("\n" + "=" * 80)
print("VCF FILE DISCOVERY (UNIFIED WORKFLOW-AWARE)")
print("=" * 80)

# Create workflow-aware discovery
discovery = VCFFileDiscovery(BASE_DIR, workflow_manager=workflow_manager)

# Single call discovers all workflows
all_workflow_vcfs = discovery.discover_all_workflows()

# Display discovered files organized by workflow
for workflow_name, stages in all_workflow_vcfs.items():
    print(f"\n{workflow_name.upper()} Workflow:")
    total_files = 0
    for stage, files in stages.items():
        if files:
            print(f"  {stage}: {len(files)} file(s)")
            total_files += len(files)
            for name, path in files.items():
                print(f"    - {name}")
    print(f"  Total: {total_files} VCF files")

print("\n" + "=" * 80)
print("DISCOVERY IMPROVEMENTS (SESSION FIXES):")
print("=" * 80)
print("✓ Fixed normalized file deduplication (removed duplicate suffix/modality keys)")
print("✓ Fixed realignment sample naming (RNA_TUMOR_realign -> RT_vs_DN format)")
print("✓ Enhanced modality name handling (DNA_TUMOR, RNA_TUMOR, DNA_NORMAL)")
print("✓ Fixed consensus file discovery (looking directly in consensus/ subdirectories)")
print("✓ Fixed annotation file discovery (cosmic_gnomad, rna_editing stages)")
print("✓ Annotation files discovered from rescue/ subdirectories (not annotation/)")
print("✓ Filtered files discovered from both filtered/ and rescue/ directories")
print("✓ Fixed BAM discovery to check both recalibrated/ and markduplicates/")
print("✓ Fixed validation summarization to handle missing improvement column")
print("✓ Fixed annotation stage detection in workflow manager")
print("✓ Fixed consensus row formatting in summary tables (Tool='consensus')")


VCF FILE DISCOVERY (UNIFIED WORKFLOW-AWARE)

STANDARD Workflow:
  normalized: 6 file(s)
    - strelka_RT_vs_DN
    - strelka_DT_vs_DN
    - deepsomatic_RT_vs_DN
    - deepsomatic_DT_vs_DN
    - mutect2_RT_vs_DN
    - mutect2_DT_vs_DN
  consensus: 2 file(s)
    - RT_vs_DN
    - DT_vs_DN
  rescue: 1 file(s)
    - DT_vs_DN_rescued_RT_vs_DN
  cosmic_gnomad: 1 file(s)
    - DT_vs_DN_rescued_RT_vs_DN
  rna_editing: 1 file(s)
    - DT_vs_DN_rescued_RT_vs_DN
  filtered_rescue: 1 file(s)
    - DT_vs_DN_rescued_RT_vs_DN
  Total: 12 VCF files

REALIGNMENT Workflow:
  normalized: 3 file(s)
    - realignment_strelka_RT_vs_DN
    - realignment_deepsomatic_RT_vs_DN
    - realignment_mutect2_RT_vs_DN
  consensus: 1 file(s)
    - realignment_RT_vs_DN
  rescue: 1 file(s)
    - realignment_DT_vs_DN_rescued_RT_vs_DN
  cosmic_gnomad: 1 file(s)
    - realignment_DT_vs_DN_rescued_RT_vs_DN
  rna_editing: 1 file(s)
    - realignment_DT_vs_DN_rescued_RT_vs_DN
  filtered_rescue: 1 file(s)
    - realignment_DT_v

In [6]:
# Discover BAM/CRAM alignment files
print("\n" + "=" * 80)
print("BAM/CRAM FILE DISCOVERY")
print("=" * 80)

# Standard BAM files
standard_bams = discovery.discover_alignments()
standard_bams = normalize_bam_files(standard_bams)

print(f"\nStandard workflow BAM files ({len(standard_bams)}):")
for modality, path in standard_bams.items():
    print(f"  {modality}: {path.name}")

# Realignment BAM files (if realignment workflow exists)
realignment_bams = {}
if WorkflowType.REALIGNMENT in detected_workflows:
    realignment_config = workflow_configs[WorkflowType.REALIGNMENT]
    raw_realignment_bams = discovery.discover_bam_files_for_workflow(realignment_config)
    
    # Normalize: strip "realignment_" prefix, then normalize suffix, then add back "_realign"
    for key, path in raw_realignment_bams.items():
        # Remove "realignment_" prefix if present
        clean_key = key.replace("realignment_", "")
        # Normalize the suffix (e.g., RNA_TUMOR_realign -> RNA_TUMOR)
        normalized = normalize_bam_key(clean_key.replace("_realign", ""))
        # Add "_realign" suffix for realignment samples
        realignment_bams[f"{normalized}_realign"] = path
    
    print(f"\nRealignment workflow BAM files ({len(realignment_bams)}):")
    for modality, path in realignment_bams.items():
        print(f"  {modality}: {path.name}")

# Combined BAM files for comprehensive validation
all_bam_files = {
    **standard_bams,
    **realignment_bams
}
print(f"\nTotal BAM files for validation: {len(all_bam_files)}")


BAM/CRAM FILE DISCOVERY

Standard workflow BAM files (3):
  DNA_TUMOR: 2374372DT.recal.cram
  RNA_TUMOR: 2374372RT.recal.cram
  DNA_NORMAL: 2374372DN.recal.cram

Realignment workflow BAM files (1):
  RNA_TUMOR_realign: 2374372RT_realign.md.cram

Total BAM files for validation: 4


## 3. VCF Statistics Processing

Process VCF files from each workflow separately using workflow-typed aggregators.

In [7]:
# Process STANDARD workflow VCFs
print("=" * 80)
print("PROCESSING STANDARD WORKFLOW VCFs")
print("=" * 80)

standard_vcfs = all_workflow_vcfs.get('standard', {})
standard_stats = process_all_vcfs(standard_vcfs)

print(f"\n✓ Processed {len(standard_stats)} stages from standard workflow")

# Create workflow-typed aggregator and visualizer
standard_aggregator = StatisticsAggregator(standard_stats, workflow_type='standard')
standard_visualizer = VCFVisualizer(standard_stats, workflow_type='standard')

print("✓ Standard workflow aggregator and visualizer created")

PROCESSING STANDARD WORKFLOW VCFs

PROCESSING: NORMALIZED

Processing: 2374372RT_vs_2374372DN.strelka.variants.dec.norm.vcf.gz
  📋 Metadata:
     Stage: normalized
     Tool: strelka
     Sample: RT_vs_DN
     File ID: strelka_RT_vs_DN
  [DEBUG] Starting header parsing...
  [DEBUG] Found 25 INFO fields in header
  [DEBUG] Processed 10001 variants, calculating statistics...
  [DEBUG] Calculated statistics for 21 INFO fields
  ✓ Total variants: 70,583
  ✓ SNPs: 66,847
  ✓ INDELs: 3,736
  ✓ Classification: {'Reference': 62094, 'Artifact': 844, 'Germline': 1183, 'Somatic': 6462}
  ✓ Chromosomes: 24

Processing: 2374372DT_vs_2374372DN.strelka.variants.dec.norm.vcf.gz
  📋 Metadata:
     Stage: normalized
     Tool: strelka
     Sample: DT_vs_DN
     File ID: strelka_DT_vs_DN
  [DEBUG] Starting header parsing...
  [DEBUG] Found 25 INFO fields in header
  [DEBUG] Processed 8297 variants, calculating statistics...
  [DEBUG] Calculated statistics for 21 INFO fields
  ✓ Total variants: 8,297
  ✓ 

In [8]:
# Process REALIGNMENT workflow VCFs (if available)
has_realignment = WorkflowType.REALIGNMENT in detected_workflows

if has_realignment:
    print("=" * 80)
    print("PROCESSING REALIGNMENT WORKFLOW VCFs")
    print("=" * 80)
    
    realignment_vcfs = all_workflow_vcfs.get('realignment', {})
    realignment_stats = process_all_vcfs(realignment_vcfs)
    
    print(f"\n✓ Processed {len(realignment_stats)} stages from realignment workflow")
    
    # Create workflow-typed aggregator and visualizer
    realignment_aggregator = StatisticsAggregator(realignment_stats, workflow_type='realignment')
    realignment_visualizer = VCFVisualizer(realignment_stats, workflow_type='realignment')
    
    print("✓ Realignment workflow aggregator and visualizer created")
else:
    print("No realignment workflow detected. Skipping realignment processing.")
    realignment_stats = None
    realignment_aggregator = None
    realignment_visualizer = None

PROCESSING REALIGNMENT WORKFLOW VCFs

PROCESSING: NORMALIZED

Processing: 2374372RT_realign_vs_2374372DN.strelka.variants.dec.norm.vcf.gz
  📋 Metadata:
     Stage: normalized
     Tool: strelka
     Sample: RT_vs_DN
     File ID: realignment_strelka_RT_vs_DN
  [DEBUG] Starting header parsing...
  [DEBUG] Found 25 INFO fields in header
  [DEBUG] Processed 10001 variants, calculating statistics...
  [DEBUG] Calculated statistics for 22 INFO fields
  ✓ Total variants: 18,309
  ✓ SNPs: 17,854
  ✓ INDELs: 455
  ✓ Classification: {'Reference': 16177, 'Somatic': 1162, 'Germline': 771, 'Artifact': 199}
  ✓ Chromosomes: 24

Processing: 2374372RT_realign_vs_2374372DN.deepsomatic.variants.dec.norm.vcf.gz
  📋 Metadata:
     Stage: normalized
     Tool: deepsomatic
     Sample: RT_vs_DN
     File ID: realignment_deepsomatic_RT_vs_DN
  [DEBUG] Starting header parsing...
  [DEBUG] Found 2 INFO fields in header
  [DEBUG] Processed 10001 variants, calculating statistics...
  [DEBUG] Calculated statisti

In [9]:
# Display detailed breakdown of all VCF categories
print("\n" + "=" * 80)
print("DETAILED VCF STATISTICS BREAKDOWN")
print("=" * 80)

def display_stats_breakdown(stats_dict, workflow_name):
    """Display statistics breakdown for a workflow with metadata."""
    print(f"\n{workflow_name.upper()} Workflow:")
    for stage, files in stats_dict.items():
        if files:
            print(f"\n  {stage} ({len(files)} files):")
            for name, data in files.items():
                total_vars = data["stats"]["basic"]["total_variants"]
                
                # Display metadata if available
                if "metadata" in data:
                    metadata = data["metadata"]
                    print(f"    {name}: {total_vars:,} variants")
                    print(f"      📋 Metadata:")
                    print(f"         Stage: {metadata.get('stage', 'N/A')}")
                    print(f"         Tool: {metadata.get('tool', 'N/A')}")
                    print(f"         Sample: {metadata.get('sample', 'N/A')}")
                    print(f"         File ID: {metadata.get('file_id', 'N/A')}")
                else:
                    print(f"    {name}: {total_vars:,} variants (no metadata)")

display_stats_breakdown(standard_stats, "Standard")

if has_realignment and realignment_stats:
    display_stats_breakdown(realignment_stats, "Realignment")



DETAILED VCF STATISTICS BREAKDOWN

STANDARD Workflow:

  normalized (6 files):
    strelka_RT_vs_DN: 70,583 variants
      📋 Metadata:
         Stage: normalized
         Tool: strelka
         Sample: RT_vs_DN
         File ID: strelka_RT_vs_DN
    strelka_DT_vs_DN: 8,297 variants
      📋 Metadata:
         Stage: normalized
         Tool: strelka
         Sample: DT_vs_DN
         File ID: strelka_DT_vs_DN
    deepsomatic_RT_vs_DN: 213,651 variants
      📋 Metadata:
         Stage: normalized
         Tool: deepsomatic
         Sample: RT_vs_DN
         File ID: deepsomatic_RT_vs_DN
    deepsomatic_DT_vs_DN: 134,025 variants
      📋 Metadata:
         Stage: normalized
         Tool: deepsomatic
         Sample: DT_vs_DN
         File ID: deepsomatic_DT_vs_DN
    mutect2_RT_vs_DN: 22,848 variants
      📋 Metadata:
         Stage: normalized
         Tool: mutect2
         Sample: RT_vs_DN
         File ID: mutect2_RT_vs_DN
    mutect2_DT_vs_DN: 4,351 variants
      📋 Metadata:
     

## 4. Statistics Aggregation and Summary Tables

In [10]:
# Generate summary tables for standard workflow
print("=" * 80)
print("STANDARD WORKFLOW SUMMARY")
print("=" * 80)

try:
    variant_summary = standard_aggregator.create_variant_count_summary()
    print("\n✓ Variant count summary:")
    display(variant_summary)
except Exception as e:
    print(f"✗ Error creating variant count summary: {e}")
    variant_summary = pd.DataFrame()

try:
    summary_report = standard_aggregator.create_summary_report()
    print("\n✓ Summary report tables:")
    for name, df in summary_report.items():
        print(f"\n{name}:")
        display(df.head(10))
except Exception as e:
    print(f"✗ Error creating summary report: {e}")
    summary_report = {}

STANDARD WORKFLOW SUMMARY

✓ Variant count summary:


,Category,Tool,Modality,File,Total_Variants,SNP,INDEL,OTHER,Somatic,Somatic_pct,Germline,Germline_pct,Reference,Reference_pct,Artifact,Artifact_pct,RNAedit,RNAedit_pct,NoConsensus,NoConsensus_pct
0,normalized,strelka,RT_vs_DN,strelka_RT_vs_DN,70583,66847,3736,0,6462,9.155179,1183.0,1.676041,62094.0,87.973025,844.0,1.195755,0.0,0.000000,0.0,0.000000
1,normalized,strelka,DT_vs_DN,strelka_DT_vs_DN,8297,8185,112,0,182,2.193564,861.0,10.377245,7197.0,86.742196,57.0,0.686995,0.0,0.000000,0.0,0.000000
2,normalized,deepsomatic,RT_vs_DN,deepsomatic_RT_vs_DN,213651,127121,86530,0,587,0.274747,6740.0,3.154677,206324.0,96.570575,0.0,0.000000,0.0,0.000000,0.0,0.000000
3,normalized,deepsomatic,DT_vs_DN,deepsomatic_DT_vs_DN,134025,131009,3016,0,91,0.067898,10226.0,7.629920,123708.0,92.302182,0.0,0.000000,0.0,0.000000,0.0,0.000000
4,normalized,mutect2,RT_vs_DN,mutect2_RT_vs_DN,22848,16898,5950,0,22848,100.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
5,normalized,mutect2,DT_vs_DN,mutect2_DT_vs_DN,4351,3225,1126,0,4351,100.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
6,consensus,consensus,RT_vs_DN,RT_vs_DN,282238,189724,92514,0,3602,1.276228,555.0,0.196643,6549.0,2.320382,8786.0,3.112976,0.0,0.000000,262746.0,93.093772
7,consensus,consensus,DT_vs_DN,DT_vs_DN,138315,134505,3810,0,150,0.108448,598.0,0.432346,5017.0,3.627228,2114.0,1.528395,0.0,0.000000,130436.0,94.303582
8,rescue,rescue,DT_vs_DN_rescued_RT_vs_DN,DT_vs_DN_rescued_RT_vs_DN,405126,309446,95680,0,3705,0.914530,1070.0,0.264115,11494.0,2.837142,10801.0,2.666084,0.0,0.000000,378056.0,93.318128
9,cosmic_gnomad,cosmic_gnomad,DT_vs_DN_rescued_RT_vs_DN,DT_vs_DN_rescued_RT_vs_DN,405125,309927,95198,0,8604,2.123789,15341.0,3.786732,10550.0,2.604135,9664.0,2.385437,0.0,0.000000,360966.0,89.099907



✓ Summary report tables:

variant_count_summary:


,Category,Tool,Modality,File,Total_Variants,SNP,INDEL,OTHER,Somatic,Somatic_pct,Germline,Germline_pct,Reference,Reference_pct,Artifact,Artifact_pct,RNAedit,RNAedit_pct,NoConsensus,NoConsensus_pct
0,normalized,strelka,RT_vs_DN,strelka_RT_vs_DN,70583,66847,3736,0,6462,9.155179,1183.0,1.676041,62094.0,87.973025,844.0,1.195755,0.0,0.0,0.0,0.000000
1,normalized,strelka,DT_vs_DN,strelka_DT_vs_DN,8297,8185,112,0,182,2.193564,861.0,10.377245,7197.0,86.742196,57.0,0.686995,0.0,0.0,0.0,0.000000
2,normalized,deepsomatic,RT_vs_DN,deepsomatic_RT_vs_DN,213651,127121,86530,0,587,0.274747,6740.0,3.154677,206324.0,96.570575,0.0,0.000000,0.0,0.0,0.0,0.000000
3,normalized,deepsomatic,DT_vs_DN,deepsomatic_DT_vs_DN,134025,131009,3016,0,91,0.067898,10226.0,7.629920,123708.0,92.302182,0.0,0.000000,0.0,0.0,0.0,0.000000
4,normalized,mutect2,RT_vs_DN,mutect2_RT_vs_DN,22848,16898,5950,0,22848,100.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
5,normalized,mutect2,DT_vs_DN,mutect2_DT_vs_DN,4351,3225,1126,0,4351,100.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
6,consensus,consensus,RT_vs_DN,RT_vs_DN,282238,189724,92514,0,3602,1.276228,555.0,0.196643,6549.0,2.320382,8786.0,3.112976,0.0,0.0,262746.0,93.093772
7,consensus,consensus,DT_vs_DN,DT_vs_DN,138315,134505,3810,0,150,0.108448,598.0,0.432346,5017.0,3.627228,2114.0,1.528395,0.0,0.0,130436.0,94.303582
8,rescue,rescue,DT_vs_DN_rescued_RT_vs_DN,DT_vs_DN_rescued_RT_vs_DN,405126,309446,95680,0,3705,0.914530,1070.0,0.264115,11494.0,2.837142,10801.0,2.666084,0.0,0.0,378056.0,93.318128
9,cosmic_gnomad,cosmic_gnomad,DT_vs_DN_rescued_RT_vs_DN,DT_vs_DN_rescued_RT_vs_DN,405125,309927,95198,0,8604,2.123789,15341.0,3.786732,10550.0,2.604135,9664.0,2.385437,0.0,0.0,360966.0,89.099907



category_distribution:


,VCF_Category,VCF_Name,Total_Variants,Somatic_pct,Somatic_count,Germline_pct,Germline_count,Reference_pct,Reference_count,Artifact_pct,Artifact_count,RNAedit_pct,RNAedit_count,NoConsensus_pct,NoConsensus_count
0,normalized,strelka_RT_vs_DN,70583,9.155179,6462,1.676041,1183,87.973025,62094,1.195755,844,0.0,0,0.000000,0
1,normalized,strelka_DT_vs_DN,8297,2.193564,182,10.377245,861,86.742196,7197,0.686995,57,0.0,0,0.000000,0
2,normalized,deepsomatic_RT_vs_DN,213651,0.274747,587,3.154677,6740,96.570575,206324,0.000000,0,0.0,0,0.000000,0
3,normalized,deepsomatic_DT_vs_DN,134025,0.067898,91,7.629920,10226,92.302182,123708,0.000000,0,0.0,0,0.000000,0
4,normalized,mutect2_RT_vs_DN,22848,100.000000,22848,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.000000,0
5,normalized,mutect2_DT_vs_DN,4351,100.000000,4351,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.000000,0
6,consensus,RT_vs_DN,282238,1.276228,3602,0.196643,555,2.320382,6549,3.112976,8786,0.0,0,93.093772,262746
7,consensus,DT_vs_DN,138315,0.108448,150,0.432346,598,3.627228,5017,1.528395,2114,0.0,0,94.303582,130436
8,rescue,DT_vs_DN_rescued_RT_vs_DN,405126,0.914530,3705,0.264115,1070,2.837142,11494,2.666084,10801,0.0,0,93.318128,378056
9,cosmic_gnomad,DT_vs_DN_rescued_RT_vs_DN,405125,2.123789,8604,3.786732,15341,2.604135,10550,2.385437,9664,0.0,0,89.099907,360966



stage_progression:


,Stage,VCF_Name,Total_Variants,SNPs,Indels,Somatic,Germline,Reference,Artifact,RNAedit,NoConsensus
0,normalized,strelka_RT_vs_DN,70583,66847,3736,6462,1183,62094,844,0,0
1,normalized,strelka_DT_vs_DN,8297,8185,112,182,861,7197,57,0,0
2,normalized,deepsomatic_RT_vs_DN,213651,127121,86530,587,6740,206324,0,0,0
3,normalized,deepsomatic_DT_vs_DN,134025,131009,3016,91,10226,123708,0,0,0
4,normalized,mutect2_RT_vs_DN,22848,16898,5950,22848,0,0,0,0,0
5,normalized,mutect2_DT_vs_DN,4351,3225,1126,4351,0,0,0,0,0
6,consensus,RT_vs_DN,282238,189724,92514,3602,555,6549,8786,0,262746
7,consensus,DT_vs_DN,138315,134505,3810,150,598,5017,2114,0,130436
8,rescue,DT_vs_DN_rescued_RT_vs_DN,405126,309446,95680,3705,1070,11494,10801,0,378056
9,cosmic_gnomad,DT_vs_DN_rescued_RT_vs_DN,405125,309927,95198,8604,15341,10550,9664,0,360966


## 5. Standard Workflow Visualizations

In [11]:
# Plot 1: Variant Counts by Tool
print("Variant Counts by Tool:")
fig = standard_visualizer.plot_variant_counts_by_tool()
if fig:
    fig.show()

Variant Counts by Tool:


In [35]:
# Plot 1: Variant Counts by Tool
print("Variant Counts by Tool:")
fig = realignment_visualizer.plot_variant_counts_by_tool()
if fig:
    fig.show()

Variant Counts by Tool:


In [12]:
# Plot 2: Variant Type Distribution
print("Variant Type Distribution:")
figs = standard_visualizer.plot_variant_type_distribution()
if figs:
    for fig in figs:
        if fig:
            fig.show()

Variant Type Distribution:


In [36]:
# Plot 2: Variant Type Distribution
print("Variant Type Distribution:")
figs = realignment_visualizer.plot_variant_type_distribution()
if figs:
    for fig in figs:
        if fig:
            fig.show()

Variant Type Distribution:


In [13]:
# Plot 3: Consensus Comparison
print("Consensus vs Individual Tools:")
fig = standard_visualizer.plot_consensus_comparison()
if fig:
    fig.show()

Consensus vs Individual Tools:


In [37]:
# Plot 3: Consensus Comparison
print("Consensus vs Individual Tools:")
fig = realignment_visualizer.plot_consensus_comparison()
if fig:
    fig.show()

Consensus vs Individual Tools:


In [14]:
# Plot 4: Filter Status
print("Filter Status:")
figs = standard_visualizer.plot_filter_status()
if figs:
    for fig in figs:
        if fig:
            fig.show()

Filter Status:


In [15]:
# Plot 5: Annotation Progression
print("Annotation Progression:")
try:
    figs = standard_visualizer.plot_annotation_progression()
    if figs:
        for fig in figs:
            if fig:
                fig.show()
except Exception as e:
    print(f"Note: Annotation progression plot not available: {e}")

Annotation Progression:


In [16]:
# Plot 5b: Full Pipeline Progression (including Consensus VCFs)
print("Full Pipeline Progression (with Consensus stages):")
try:
    figs = standard_visualizer.plot_full_pipeline_progression(
        include_consensus=True,
        dual_view_no_consensus=True,
        workflow_type="Standard"
    )
    if figs:
        for fig in figs:
            if fig:
                fig.show()
except Exception as e:
    print(f"Note: Full pipeline progression plot not available: {e}")

Full Pipeline Progression (with Consensus stages):


In [39]:
# Plot 5b: Full Pipeline Progression (including Consensus VCFs)
print("Full Pipeline Progression (with Consensus stages):")
try:
    figs = realignment_visualizer.plot_full_pipeline_progression(
        include_consensus=True,
        dual_view_no_consensus=True,
        workflow_type="Realignment"
    )
    if figs:
        for fig in figs:
            if fig:
                fig.show()
except Exception as e:
    print(f"Note: Full pipeline progression plot not available: {e}")

Full Pipeline Progression (with Consensus stages):


In [17]:
# Plot 6: Category Heatmap
print("Category Heatmap:")
figs = standard_visualizer.plot_category_heatmap()
if figs:
    for fig in figs:
        if fig:
            fig.show()

Category Heatmap:


In [18]:
# Plot 7: Stage Transition Sankey
print("Stage Transition Sankey Diagram:")
fig = standard_visualizer.plot_stage_transition_sankey(
    categories_to_show=["Somatic", "Germline", "RNAedit", "NoConsensus"]
)
if fig:
    fig.show()

Stage Transition Sankey Diagram:


In [19]:
# Plot 8: Tier Distribution (CxDy Hybrid System)
print("=" * 80)
print("TIER DISTRIBUTION (CxDy: Caller Support × Database Evidence)")
print("=" * 80)

# First, get tiering statistics manually
from vcf_stats import tier_rescue_variants
from tier_config import TIER_ORDER, get_tier_quality

# Find rescue VCF from standard_stats - always use filtered_rescue
rescue_path = None
if 'filtered_rescue' in standard_stats and standard_stats['filtered_rescue']:
    first_entry = next(iter(standard_stats['filtered_rescue'].values()))
    if isinstance(first_entry, dict) and 'path' in first_entry:
        rescue_path = first_entry['path']
    elif hasattr(first_entry, 'name'):  # It's a Path
        rescue_path = first_entry
    print("Using VCF from stage: filtered_rescue")
else:
    print("Error: filtered_rescue stage not found - tier distribution requires filtered rescue VCF")

if rescue_path:
    print(f"Rescue VCF: {rescue_path.name}")
    
    tiered_df = tier_rescue_variants(rescue_path)
    print(f"Total variants: {len(tiered_df):,}")
    
    # Print category breakdown with tier distribution
    print("\n" + "-" * 60)
    print("TIERING STATISTICS BY CATEGORY")
    print("-" * 60)
    
    for category in CATEGORY_ORDER:
        cat_data = tiered_df[tiered_df['filter_category'] == category]
        if len(cat_data) > 0:
            pct = (len(cat_data) / len(tiered_df)) * 100
            print(f"\n{category}: {len(cat_data):,} variants ({pct:.1f}%)")
            tier_counts = cat_data['tier'].value_counts().sort_index()
            for tier in TIER_ORDER:
                if tier in tier_counts.index:
                    count = tier_counts[tier]
                    tier_pct = (count / len(cat_data)) * 100
                    print(f"  {tier}: {count:,} ({tier_pct:.1f}%)")
    
    # Print D0 vs D1 summary
    d0_count = len(tiered_df[tiered_df['tier'].str.endswith('D0')])
    d1_count = len(tiered_df[tiered_df['tier'].str.endswith('D1')])
    print("\n" + "-" * 60)
    print("DATABASE SUPPORT SUMMARY (D0 vs D1)")
    print("-" * 60)
    print(f"D0 (no database support): {d0_count:,} ({(d0_count/len(tiered_df))*100:.1f}%)")
    print(f"D1 (has database support): {d1_count:,} ({(d1_count/len(tiered_df))*100:.1f}%)")
    
    # D0 vs D1 by category
    print("\nD0 vs D1 breakdown by category:")
    for category in CATEGORY_ORDER:
        cat_data = tiered_df[tiered_df['filter_category'] == category]
        if len(cat_data) > 0:
            cat_d0 = len(cat_data[cat_data['tier'].str.endswith('D0')])
            cat_d1 = len(cat_data[cat_data['tier'].str.endswith('D1')])
            print(f"  {category}: D0={cat_d0:,}, D1={cat_d1:,}")
    
    # Print caller tier summary
    print("\n" + "-" * 60)
    print("CALLER SUPPORT SUMMARY (C1-C7)")
    print("-" * 60)
    for i in range(1, 8):
        c_tier = f"C{i}"
        c_count = len(tiered_df[tiered_df['tier'].str.startswith(c_tier)])
        if c_count > 0:
            pct = (c_count / len(tiered_df)) * 100
            print(f"{c_tier}: {c_count:,} ({pct:.1f}%)")
else:
    print("No rescue VCF found in standard_stats")

print("\n" + "=" * 80)
print("GENERATING TIER DISTRIBUTION PLOTS")
print("=" * 80)

figs = standard_visualizer.plot_tier_distribution()
if figs:
    plot_names = [
        "Plot 1: All categories - Full tier distribution (stacked by category)",
        "Plot 2: Excluding NoConsensus - Focused view (stacked by category)",
        "Plot 3: D0 vs D1 comparison by category (grouped bars)"
    ]
    for i, fig in enumerate(figs):
        if fig:
            name = plot_names[i] if i < len(plot_names) else f"Plot {i+1}"
            print(f"\n{name}")
            if i < 2:
                print("  X-axis: Individual tiers (C1D0, C2D0, C2D1, C3D0, C3D1, ...)")
                print("  Each bar shows stacked counts per category")
                print("  Legend: All 6 categories including RNAedit")
            fig.show()
else:
    print("No tier distribution plots generated")

TIER DISTRIBUTION (CxDy: Caller Support × Database Evidence)
Using VCF from stage: filtered_rescue
Rescue VCF: 2374372DT_vs_2374372DN_rescued_2374372RT_vs_2374372DN.filtered.vcf.gz


Total variants: 405,125

------------------------------------------------------------
TIERING STATISTICS BY CATEGORY
------------------------------------------------------------

Somatic: 8,392 variants (2.1%)
  C1D1: 16 (0.2%)
  C1D0: 24 (0.3%)
  C2D1: 42 (0.5%)
  C2D0: 65 (0.8%)
  C3D1: 824 (9.8%)
  C3D0: 2,492 (29.7%)
  C4D1: 74 (0.9%)
  C4D0: 4 (0.0%)
  C5D1: 432 (5.1%)
  C5D0: 177 (2.1%)
  C6D1: 51 (0.6%)
  C7D1: 4,191 (49.9%)

Germline: 15,326 variants (3.8%)
  C1D1: 16 (0.1%)
  C1D0: 5 (0.0%)
  C2D1: 315 (2.1%)
  C2D0: 68 (0.4%)
  C3D1: 149 (1.0%)
  C3D0: 60 (0.4%)
  C4D1: 1,256 (8.2%)
  C4D0: 314 (2.0%)
  C5D1: 3,521 (23.0%)
  C5D0: 605 (3.9%)
  C6D1: 410 (2.7%)
  C6D0: 375 (2.4%)
  C7D1: 4,652 (30.4%)
  C7D0: 3,580 (23.4%)

Reference: 10,137 variants (2.5%)
  C1D1: 8 (0.1%)
  C1D0: 19 (0.2%)
  C2D1: 769 (7.6%)
  C2D0: 3,440 (33.9%)
  C3D1: 992 (9.8%)
  C3D0: 4,909 (48.4%)

Artifact: 9,401 variants (2.3%)
  C4D0: 1 (0.0%)
  C5D1: 1 (0.0%)
  C5D0: 13 (0.1%)
  C6D1: 9 (0.1%)
  C6


Plot 2: Excluding NoConsensus - Focused view (stacked by category)
  X-axis: Individual tiers (C1D0, C2D0, C2D1, C3D0, C3D1, ...)
  Each bar shows stacked counts per category
  Legend: All 6 categories including RNAedit



Plot 3: D0 vs D1 comparison by category (grouped bars)


In [38]:
# Plot 8: Tier Distribution for REALIGNMENT Workflow (CxDy Hybrid System)
if has_realignment:
    print("=" * 80)
    print("REALIGNMENT WORKFLOW - TIER DISTRIBUTION (CxDy: Caller Support × Database Evidence)")
    print("=" * 80)

    # First, get tiering statistics manually
    from vcf_stats import tier_rescue_variants
    from tier_config import TIER_ORDER, get_tier_quality

    # Find rescue VCF from realignment_stats - always use filtered_rescue
    rescue_path = None
    if 'filtered_rescue' in realignment_stats and realignment_stats['filtered_rescue']:
        first_entry = next(iter(realignment_stats['filtered_rescue'].values()))
        if isinstance(first_entry, dict) and 'path' in first_entry:
            rescue_path = first_entry['path']
        elif hasattr(first_entry, 'name'):  # It's a Path
            rescue_path = first_entry
        print("Using VCF from stage: filtered_rescue (realignment)")
    else:
        print("Error: filtered_rescue stage not found - tier distribution requires filtered rescue VCF")

    if rescue_path:
        print(f"Rescue VCF: {rescue_path.name}")
        
        tiered_df = tier_rescue_variants(rescue_path)
        print(f"Total variants: {len(tiered_df):,}")
        
        # Print category breakdown with tier distribution
        print("\n" + "-" * 60)
        print("TIERING STATISTICS BY CATEGORY (REALIGNMENT)")
        print("-" * 60)
        
        for category in CATEGORY_ORDER:
            cat_data = tiered_df[tiered_df['filter_category'] == category]
            if len(cat_data) > 0:
                pct = (len(cat_data) / len(tiered_df)) * 100
                print(f"\n{category}: {len(cat_data):,} variants ({pct:.1f}%)")
                tier_counts = cat_data['tier'].value_counts().sort_index()
                for tier in TIER_ORDER:
                    if tier in tier_counts.index:
                        count = tier_counts[tier]
                        tier_pct = (count / len(cat_data)) * 100
                        print(f"  {tier}: {count:,} ({tier_pct:.1f}%)")
        
        # Print D0 vs D1 summary
        d0_count = len(tiered_df[tiered_df['tier'].str.endswith('D0')])
        d1_count = len(tiered_df[tiered_df['tier'].str.endswith('D1')])
        print("\n" + "-" * 60)
        print("DATABASE SUPPORT SUMMARY (D0 vs D1) - REALIGNMENT")
        print("-" * 60)
        print(f"D0 (no database support): {d0_count:,} ({(d0_count/len(tiered_df))*100:.1f}%)")
        print(f"D1 (has database support): {d1_count:,} ({(d1_count/len(tiered_df))*100:.1f}%)")
        
        # D0 vs D1 by category
        print("\nD0 vs D1 breakdown by category:")
        for category in CATEGORY_ORDER:
            cat_data = tiered_df[tiered_df['filter_category'] == category]
            if len(cat_data) > 0:
                cat_d0 = len(cat_data[cat_data['tier'].str.endswith('D0')])
                cat_d1 = len(cat_data[cat_data['tier'].str.endswith('D1')])
                print(f"  {category}: D0={cat_d0:,}, D1={cat_d1:,}")
        
        # Print caller tier summary
        print("\n" + "-" * 60)
        print("CALLER SUPPORT SUMMARY (C1-C7) - REALIGNMENT")
        print("-" * 60)
        for i in range(1, 8):
            c_tier = f"C{i}"
            c_count = len(tiered_df[tiered_df['tier'].str.startswith(c_tier)])
            if c_count > 0:
                pct = (c_count / len(tiered_df)) * 100
                print(f"{c_tier}: {c_count:,} ({pct:.1f}%)")
    else:
        print("No rescue VCF found in realignment_stats")

    print("\n" + "=" * 80)
    print("GENERATING TIER DISTRIBUTION PLOTS (REALIGNMENT)")
    print("=" * 80)

    figs = realignment_visualizer.plot_tier_distribution()
    if figs:
        plot_names = [
            "Plot 1: All categories - Full tier distribution (stacked by category)",
            "Plot 2: Excluding NoConsensus - Focused view (stacked by category)",
            "Plot 3: D0 vs D1 comparison by category (grouped bars)"
        ]
        for i, fig in enumerate(figs):
            if fig:
                name = plot_names[i] if i < len(plot_names) else f"Plot {i+1}"
                print(f"\n{name}")
                if i < 2:
                    print("  X-axis: Individual tiers (C1D0, C2D0, C2D1, C3D0, C3D1, ...)")
                    print("  Each bar shows stacked counts per category")
                    print("  Legend: All 6 categories including RNAedit")
                fig.show()
    else:
        print("No tier distribution plots generated for realignment workflow")
else:
    print("=" * 80)
    print("REALIGNMENT WORKFLOW - TIER DISTRIBUTION")
    print("=" * 80)
    print("⚠️  Skipping realignment tier distribution (no realignment data available)")

REALIGNMENT WORKFLOW - TIER DISTRIBUTION (CxDy: Caller Support × Database Evidence)
Using VCF from stage: filtered_rescue (realignment)
Rescue VCF: 2374372DT_vs_2374372DN_rescued_2374372RT_realign_vs_2374372DN.filtered.vcf.gz
Total variants: 171,158

------------------------------------------------------------
TIERING STATISTICS BY CATEGORY (REALIGNMENT)
------------------------------------------------------------

Somatic: 5,649 variants (3.3%)
  C1D1: 15 (0.3%)
  C1D0: 21 (0.4%)
  C2D1: 43 (0.8%)
  C2D0: 69 (1.2%)
  C3D1: 405 (7.2%)
  C3D0: 665 (11.8%)
  C4D1: 19 (0.3%)
  C4D0: 2 (0.0%)
  C5D1: 452 (8.0%)
  C5D0: 180 (3.2%)
  C6D1: 18 (0.3%)
  C7D1: 3,760 (66.6%)

Germline: 14,723 variants (8.6%)
  C1D1: 13 (0.1%)
  C1D0: 7 (0.0%)
  C2D1: 329 (2.2%)
  C2D0: 66 (0.4%)
  C3D1: 103 (0.7%)
  C3D0: 40 (0.3%)
  C4D1: 1,194 (8.1%)
  C4D0: 270 (1.8%)
  C5D1: 3,915 (26.6%)
  C5D0: 659 (4.5%)
  C6D1: 293 (2.0%)
  C6D0: 284 (1.9%)
  C7D1: 4,344 (29.5%)
  C7D0: 3,206 (21.8%)

Reference: 8,226 va


Plot 2: Excluding NoConsensus - Focused view (stacked by category)
  X-axis: Individual tiers (C1D0, C2D0, C2D1, C3D0, C3D1, ...)
  Each bar shows stacked counts per category
  Legend: All 6 categories including RNAedit



Plot 3: D0 vs D1 comparison by category (grouped bars)


## 6. Rescue VCF Analysis

In [20]:
# Analyze rescue VCFs
print("=" * 80)
print("RESCUE VCF ANALYSIS")
print("=" * 80)

analyze_rescue_vcf(standard_stats)

RESCUE VCF ANALYSIS

Category        DNA Consensus   RNA Consensus   Rescued         COSMIC          RNA_Edit        Filtered       
------------------------------------------------------------
Somatic         150             3,602           3,705           8,604           8,392           8,392          
Germline        598             555             1,070           15,341          15,326          15,326         
Reference       5,017           6,549           11,494          10,550          10,137          10,137         
Artifact        2,114           8,786           10,801          9,664           9,401           9,401          
RNAedit         0               0               0               0               903             903            
NoConsensus     130,436         262,746         378,056         360,966         360,966         360,966        
PASS            0               0               0               0               0               0              
LowQual         0     

{'dna_consensus': {'total_variants': 138315,
  'snps': 134505,
  'indels': 3810,
  'mnps': 0,
  'complex': 0,
  'chromosomes': ['chr1',
   'chr10',
   'chr11',
   'chr12',
   'chr13',
   'chr14',
   'chr15',
   'chr16',
   'chr17',
   'chr18',
   'chr19',
   'chr2',
   'chr20',
   'chr21',
   'chr22',
   'chr3',
   'chr4',
   'chr5',
   'chr6',
   'chr7',
   'chr8',
   'chr9',
   'chrM',
   'chrX'],
  'variant_types': defaultdict(int, {'SNP': 134505, 'DEL': 2603, 'INS': 1207}),
  'category_variant_types': {'NoConsensus': {'SNP': 127047,
    'INDEL': 3389,
    'OTHER': 0},
   'Artifact': {'SNP': 1762, 'INDEL': 352, 'OTHER': 0},
   'Reference': {'SNP': 4961, 'INDEL': 56, 'OTHER': 0},
   'Somatic': {'SNP': 148, 'INDEL': 2, 'OTHER': 0},
   'Germline': {'SNP': 587, 'INDEL': 11, 'OTHER': 0}},
  'classification': {'NoConsensus': 130436,
   'Artifact': 2114,
   'Reference': 5017,
   'Somatic': 150,
   'Germline': 598},
  'category_distribution': {'NoConsensus': 94.30358240248708,
   'Artifact'

## 7. Workflow Comparison: Standard vs Realignment

**Purpose**: Compare RNA variant calling between standard and realignment workflows to assess realignment effectiveness.

**Context**: The realignment workflow only applies to RNA samples. It realigns RNA reads around DNA consensus variants to improve RNA variant calling accuracy by reducing false negatives at known somatic sites.

**Comparison Analysis:**
1. **Variant Count Comparison**: Total variants by stage
2. **Category Distribution**: Changes in Somatic, Germline, Artifact classifications
3. **Annotation Impact**: Database annotation effects (COSMIC, gnomAD, RNA editing)
4. **Integrative View**: DNA + RNA Standard + RNA Realignment combined statistics
5. **Realignment Impact**: Quantitative improvement metrics

In [21]:
# Workflow Comparison: Standard vs Realignment
if has_realignment:
    print("=" * 80)
    print("WORKFLOW COMPARISON: STANDARD vs REALIGNMENT")
    print("=" * 80)
    
    # Create comparator with full stats
    comparator = WorkflowComparator(standard_stats, realignment_stats)
    
    # 1. RNA Variant Count Comparison
    print("\n" + "=" * 80)
    print("1. RNA VARIANT COUNT COMPARISON")
    print("=" * 80)
    rna_count_comparison = comparator.compare_rna_variant_counts()
    display(rna_count_comparison)
    
    # 2. RNA Category Distribution Comparison
    print("\n" + "=" * 80)
    print("2. RNA CATEGORY DISTRIBUTION COMPARISON")
    print("=" * 80)
    rna_category_comparison = comparator.compare_rna_category_distribution()
    display(rna_category_comparison.head(20))
    
    # 3. RNA Annotation Stage Comparison
    print("\n" + "=" * 80)
    print("3. RNA ANNOTATION STAGE COMPARISON")
    print("=" * 80)
    rna_annotation_comparison = comparator.compare_rna_annotation_stages()
    display(rna_annotation_comparison)
    
    # 4. Integrative View: DNA + RNA Standard + RNA Realignment
    print("\n" + "=" * 80)
    print("4. INTEGRATIVE VIEW: DNA + RNA STANDARD + RNA REALIGNMENT")
    print("=" * 80)
    integrative_view = comparator.create_integrative_view()
    display(integrative_view)
    
    # 5. Realignment Impact Summary
    print("\n" + "=" * 80)
    print("5. REALIGNMENT IMPACT SUMMARY")
    print("=" * 80)
    realignment_impact = comparator.calculate_realignment_impact()
    
    print(f"📊 Quantitative Metrics:")
    print(f"   RNA variants added: {realignment_impact.get('rna_variants_added', 0):,}")
    print(f"   RNA variants removed: {realignment_impact.get('rna_variants_removed', 0):,}")
    print(f"   Net improvement: {realignment_impact.get('realignment_improvement', 0):.2f}%")
    
    if 'category_changes' in realignment_impact:
        print(f"\n📈 Category Changes:")
        for category, delta in realignment_impact['category_changes'].items():
            symbol = "+" if delta > 0 else ""
            print(f"   {category}: {symbol}{delta:,}")
    
    print("\n✅ Workflow comparison complete")
else:
    print("⚠️  Realignment workflow not detected - skipping comparison")
    print("   Only standard workflow will be analyzed")
    comparator = None

WORKFLOW COMPARISON: STANDARD vs REALIGNMENT

1. RNA VARIANT COUNT COMPARISON


,Stage,Standard_RNA_Count,Realignment_RNA_Count,Difference,Percent_Change
0,normalized,307082,52908,-254174,-82.770726
1,consensus,282238,43496,-238742,-84.588893
2,rescue,405126,171159,-233967,-57.751662
3,cosmic_gnomad,405125,171158,-233967,-57.751805
4,rna_editing,405125,171158,-233967,-57.751805
5,filtered_rescue,405125,171158,-233967,-57.751805



2. RNA CATEGORY DISTRIBUTION COMPARISON


,Stage,Category,Standard_RNA_Count,Realignment_RNA_Count,Difference,Percent_Change
0,normalized,Somatic,29897,7405,-22492,-75.231629
1,normalized,Germline,7923,6571,-1352,-17.064243
2,normalized,Reference,268418,38733,-229685,-85.569895
3,normalized,Artifact,844,199,-645,-76.421801
4,consensus,Somatic,3602,1237,-2365,-65.657968
5,consensus,Germline,555,382,-173,-31.171171
6,consensus,Reference,6549,4247,-2302,-35.150405
7,consensus,Artifact,8786,2232,-6554,-74.595948
8,consensus,NoConsensus,262746,35398,-227348,-86.527673
9,rescue,Somatic,3705,1345,-2360,-63.697706



3. RNA ANNOTATION STAGE COMPARISON


,Stage,Category,Standard_RNA_Count,Realignment_RNA_Count,Difference,Percent_Change
0,cosmic_gnomad,Somatic,8604,5767,-2837,-32.973036
1,cosmic_gnomad,Germline,15341,14725,-616,-4.015384
2,cosmic_gnomad,Reference,10550,8372,-2178,-20.644550
3,cosmic_gnomad,Artifact,9664,3304,-6360,-65.811258
4,cosmic_gnomad,NoConsensus,360966,138990,-221976,-61.494988
5,rna_editing,Somatic,8392,5649,-2743,-32.685891
6,rna_editing,Germline,15326,14723,-603,-3.934490
7,rna_editing,Reference,10137,8226,-1911,-18.851731
8,rna_editing,Artifact,9401,3175,-6226,-66.226997
9,rna_editing,RNAedit,903,395,-508,-56.256921



4. INTEGRATIVE VIEW: DNA + RNA STANDARD + RNA REALIGNMENT


,Stage,Category,DNA_Standard,RNA_Standard,RNA_Realignment,RNA_Difference
0,normalized,Somatic,4624,29897,7405,-22492
1,normalized,Germline,11087,7923,6571,-1352
2,normalized,Reference,130905,268418,38733,-229685
3,normalized,Artifact,57,844,199,-645
4,consensus,Somatic,150,3602,1237,-2365
5,consensus,Germline,598,555,382,-173
6,consensus,Reference,5017,6549,4247,-2302
7,consensus,Artifact,2114,8786,2232,-6554
8,consensus,NoConsensus,130436,262746,35398,-227348
9,rescue,Somatic,0,3705,1345,-2360



5. REALIGNMENT IMPACT SUMMARY
📊 Quantitative Metrics:
   RNA variants added: 0
   RNA variants removed: 233,967
   Net improvement: -57.75%

📈 Category Changes:
   NoConsensus: -221,976
   RNAedit: -508
   Reference: -1,911
   Artifact: -6,226
   Germline: -603
   Somatic: -2,743

✅ Workflow comparison complete


## 8. Workflow Comparison Visualizations

Visual comparison of standard vs realignment workflows across multiple dimensions.

In [22]:
# Workflow Comparison Visualizations
if has_realignment:
    print("=" * 80)
    print("WORKFLOW COMPARISON VISUALIZATIONS")
    print("=" * 80)
    
    # Extract RNA-specific stats from standard workflow (used by multiple plots)
    rna_standard_stats = {}
    for stage, files in standard_stats.items():
        rna_files = {k: v for k, v in files.items() if 'RT' in k or 'RNA' in k}
        if rna_files:
            rna_standard_stats[stage] = rna_files
    
    # Extract DNA-specific stats (used by integrative plot)
    dna_standard_stats = {}
    for stage, files in standard_stats.items():
        dna_files = {k: v for k, v in files.items() if 'DT' in k or 'DNA_TUMOR' in k}
        if dna_files:
            dna_standard_stats[stage] = dna_files
    
    # Plot 1: RNA Workflow Comparison
    print("\n📊 Plot 1: RNA Workflow Comparison (Standard vs Realignment)")
    try:
        fig = standard_visualizer.plot_rna_workflow_comparison(
            rna_standard_stats,
            realignment_stats
        )
        fig.show()
    except Exception as e:
        print(f"   ⚠️  Could not generate plot: {e}")
    
    # Plot 2: RNA Stage Progression Comparison
    print("\n📊 Plot 2: RNA Stage Progression Comparison")
    try:
        fig = standard_visualizer.plot_rna_stage_progression_comparison(
            rna_standard_stats,
            realignment_stats
        )
        fig.show()
    except Exception as e:
        print(f"   ⚠️  Could not generate plot: {e}")
    
    # Plot 3: RNA Annotation Impact Heatmap
    print("\n📊 Plot 3: RNA Annotation Impact Heatmap")
    try:
        fig = standard_visualizer.plot_rna_annotation_impact_comparison(
            rna_standard_stats,
            realignment_stats
        )
        fig.show()
    except Exception as e:
        print(f"   ⚠️  Could not generate plot: {e}")
    
    # Plot 4: Integrative View (DNA + RNA Standard + RNA Realignment)
    print("\n📊 Plot 4: Integrative View (DNA + RNA Standard + RNA Realignment)")
    try:
        fig = standard_visualizer.plot_integrative_view(
            dna_standard_stats,
            rna_standard_stats,
            realignment_stats
        )
        fig.show()
    except Exception as e:
        print(f"   ⚠️  Could not generate plot: {e}")
    
    print("\n✅ Generated 4 workflow comparison visualizations")
else:
    print("⚠️  Skipping workflow comparison visualizations (no realignment data)")

WORKFLOW COMPARISON VISUALIZATIONS

📊 Plot 1: RNA Workflow Comparison (Standard vs Realignment)



📊 Plot 2: RNA Stage Progression Comparison



📊 Plot 3: RNA Annotation Impact Heatmap



📊 Plot 4: Integrative View (DNA + RNA Standard + RNA Realignment)



✅ Generated 4 workflow comparison visualizations


## 8.5. Caller Comparison with Venn Diagrams

Compare variant calling results across Strelka, DeepSomatic, and Mutect2 using:
- **Focused 3-way Venn diagrams**: 4 plots per comparison
  - Somatic SNP (high-confidence cancer mutations)
  - Somatic INDEL (high-confidence cancer indels)
  - All SNP (aggregated across all categories)
  - All INDEL (aggregated across all categories)
- **Modality-specific comparison**: DNA callers, RNA callers (standard and realignment)
- **3-way consensus comparison**: DNA standard vs RNA standard vs RNA realignment (all categories)
- **Interactive plotly visualizations**: HTML-embeddable Venn diagrams
- **Consistent color scheme**: Red (Strelka), Blue (DeepSomatic), Green (Mutect2)

**Key Features:**
- Uses `classify_by_filter()` to classify normalized VCF variants
- Variants keyed by (CHROM, POS, REF, ALT, FILTER_CATEGORY)
- Shows caller agreement for **Somatic variants** (most clinically relevant)
- Shows overall caller agreement across **all categories**
- Identifies high-confidence variants (supported by multiple callers)

**Comparison Structure:**
1. DNA callers (standard): 4 Venn diagrams (Somatic SNP/INDEL + All SNP/INDEL)
2. RNA callers (standard): 4 Venn diagrams (Somatic SNP/INDEL + All SNP/INDEL)
3. RNA callers (realignment): 4 Venn diagrams (Somatic SNP/INDEL + All SNP/INDEL)
4. Consensus 3-way: DNA vs RNA std vs RNA realign (per-category for all categories)

In [23]:
# Create caller comparator
print("=" * 80)
print("CALLER COMPARISON INITIALIZATION")
print("=" * 80)

# Initialize comparator with all discovered VCFs
caller_comparator = CallerComparator(all_workflow_vcfs)

print("✓ CallerComparator initialized")
print(f"  Available workflows: {list(all_workflow_vcfs.keys())}")
print(f"  Ready for DNA and RNA caller comparison")

CALLER COMPARISON INITIALIZATION
✓ CallerComparator initialized
  Available workflows: ['standard', 'realignment']
  Ready for DNA and RNA caller comparison


In [24]:
# DNA Caller Comparison (Standard Workflow) - Focused Venn Diagrams
print("=" * 80)
print("DNA CALLER COMPARISON (Standard Workflow)")
print("=" * 80)
print("Generating 4 focused Venn diagrams:")
print("  1. Somatic SNP (high-confidence cancer mutations)")
print("  2. Somatic INDEL (high-confidence cancer indels)")
print("  3. All SNP (across all categories)")
print("  4. All INDEL (across all categories)")
print()

try:
    dna_summary, dna_figures = caller_comparator.compare_dna_callers(
        workflow="standard",
        stage="normalized"
    )
    
    print("\n✓ DNA Caller Comparison Summary:")
    display(dna_summary)
    
    print(f"\n✓ Generated {len(dna_figures)} Venn diagrams")
    
    # Display Venn diagrams in focused order
    if dna_figures:
        print("\nDNA Focused Venn Diagrams:")
        # Show in logical order: Somatic first, then All
        display_order = ["Somatic_SNP", "Somatic_INDEL", "All_SNP", "All_INDEL"]
        for suffix in display_order:
            for fig_key in sorted(dna_figures.keys()):
                if fig_key.endswith(suffix):
                    print(f"\n{fig_key}:")
                    dna_figures[fig_key].show()
        
except Exception as e:
    print(f"✗ Error in DNA caller comparison: {e}")
    import traceback
    traceback.print_exc()
    dna_summary = None
    dna_figures = {}

DNA CALLER COMPARISON (Standard Workflow)
Generating 4 focused Venn diagrams:
  1. Somatic SNP (high-confidence cancer mutations)
  2. Somatic INDEL (high-confidence cancer indels)
  3. All SNP (across all categories)
  4. All INDEL (across all categories)

Found DNA callers: ['strelka', 'deepsomatic', 'mutect2']
  Extracting variants from strelka...
  Extracting variants from deepsomatic...
  Extracting variants from mutect2...
  Generating focused Venn diagrams (Somatic + All categories)...
  Generating Somatic SNP and INDEL Venn diagrams...
  Generating All-categories SNP and INDEL Venn diagrams...

✓ DNA Caller Comparison Summary:


,Comparison,Category,Variant_Type,strelka_count,deepsomatic_count,mutect2_count,All_Three,Any_Two_or_More
0,DNA_standard_normalized,Somatic,SNP,178,89,3225,79,148
1,DNA_standard_normalized,Somatic,INDEL,4,2,1126,1,2
2,DNA_standard_normalized,All,SNP,8185,131009,3225,79,5696
3,DNA_standard_normalized,All,INDEL,112,3016,1126,1,69



✓ Generated 4 Venn diagrams

DNA Focused Venn Diagrams:

DNA_standard_normalized_Somatic_SNP:



DNA_standard_normalized_Somatic_INDEL:



DNA_standard_normalized_All_SNP:



DNA_standard_normalized_All_INDEL:


In [25]:
# RNA Caller Comparison (Standard Workflow) - Focused Venn Diagrams
print("=" * 80)
print("RNA CALLER COMPARISON (Standard Workflow)")
print("=" * 80)
print("Generating 4 focused Venn diagrams:")
print("  1. Somatic SNP (high-confidence cancer mutations)")
print("  2. Somatic INDEL (high-confidence cancer indels)")
print("  3. All SNP (across all categories)")
print("  4. All INDEL (across all categories)")
print()

try:
    rna_summary, rna_figures = caller_comparator.compare_rna_callers(
        workflow="standard",
        stage="normalized"
    )
    
    print("\n✓ RNA Caller Comparison Summary:")
    display(rna_summary)
    
    print(f"\n✓ Generated {len(rna_figures)} Venn diagrams")
    
    # Display Venn diagrams in focused order
    if rna_figures:
        print("\nRNA Focused Venn Diagrams:")
        # Show in logical order: Somatic first, then All
        display_order = ["Somatic_SNP", "Somatic_INDEL", "All_SNP", "All_INDEL"]
        for suffix in display_order:
            for fig_key in sorted(rna_figures.keys()):
                if fig_key.endswith(suffix):
                    print(f"\n{fig_key}:")
                    rna_figures[fig_key].show()
        
except Exception as e:
    print(f"✗ Error in RNA caller comparison: {e}")
    import traceback
    traceback.print_exc()
    rna_summary = None
    rna_figures = {}

RNA CALLER COMPARISON (Standard Workflow)
Generating 4 focused Venn diagrams:
  1. Somatic SNP (high-confidence cancer mutations)
  2. Somatic INDEL (high-confidence cancer indels)
  3. All SNP (across all categories)
  4. All INDEL (across all categories)

Found RNA callers: ['strelka', 'deepsomatic', 'mutect2']
  Extracting variants from strelka...


  Extracting variants from deepsomatic...
  Extracting variants from mutect2...
  Generating focused Venn diagrams (Somatic + All categories)...
  Generating Somatic SNP and INDEL Venn diagrams...
  Generating All-categories SNP and INDEL Venn diagrams...

✓ RNA Caller Comparison Summary:


,Comparison,Category,Variant_Type,strelka_count,deepsomatic_count,mutect2_count,All_Three,Any_Two_or_More
0,RNA_standard_normalized,Somatic,SNP,5689,248,16898,192,2875
1,RNA_standard_normalized,Somatic,INDEL,773,339,5950,186,727
2,RNA_standard_normalized,All,SNP,66847,127121,16898,192,9437
3,RNA_standard_normalized,All,INDEL,3736,86529,5950,186,1269



✓ Generated 4 Venn diagrams

RNA Focused Venn Diagrams:

RNA_standard_normalized_Somatic_SNP:



RNA_standard_normalized_Somatic_INDEL:



RNA_standard_normalized_All_SNP:



RNA_standard_normalized_All_INDEL:


In [26]:
# RNA Caller Comparison (Realignment Workflow) - Focused Venn Diagrams - if available
if has_realignment:
    print("=" * 80)
    print("RNA CALLER COMPARISON (Realignment Workflow)")
    print("=" * 80)
    print("Generating 4 focused Venn diagrams:")
    print("  1. Somatic SNP (high-confidence cancer mutations)")
    print("  2. Somatic INDEL (high-confidence cancer indels)")
    print("  3. All SNP (across all categories)")
    print("  4. All INDEL (across all categories)")
    print()
    
    try:
        rna_realign_summary, rna_realign_figures = caller_comparator.compare_rna_callers(
            workflow="realignment",
            stage="normalized"
        )
        
        print("\n✓ RNA Realignment Caller Comparison Summary:")
        display(rna_realign_summary)
        
        print(f"\n✓ Generated {len(rna_realign_figures)} Venn diagrams")
        
        # Display Venn diagrams in focused order
        if rna_realign_figures:
            print("\nRNA Realignment Focused Venn Diagrams:")
            # Show in logical order: Somatic first, then All
            display_order = ["Somatic_SNP", "Somatic_INDEL", "All_SNP", "All_INDEL"]
            for suffix in display_order:
                for fig_key in sorted(rna_realign_figures.keys()):
                    if fig_key.endswith(suffix):
                        print(f"\n{fig_key}:")
                        rna_realign_figures[fig_key].show()
            
    except Exception as e:
        print(f"✗ Error in RNA realignment caller comparison: {e}")
        import traceback
        traceback.print_exc()
        rna_realign_summary = None
        rna_realign_figures = {}
else:
    print("Skipping RNA realignment caller comparison (no realignment data).")

RNA CALLER COMPARISON (Realignment Workflow)
Generating 4 focused Venn diagrams:
  1. Somatic SNP (high-confidence cancer mutations)
  2. Somatic INDEL (high-confidence cancer indels)
  3. All SNP (across all categories)
  4. All INDEL (across all categories)

Found RNA callers: ['strelka', 'deepsomatic', 'mutect2']
  Extracting variants from strelka...
  Extracting variants from deepsomatic...
  Extracting variants from mutect2...
  Generating focused Venn diagrams (Somatic + All categories)...
  Generating Somatic SNP and INDEL Venn diagrams...
  Generating All-categories SNP and INDEL Venn diagrams...

✓ RNA Realignment Caller Comparison Summary:


,Comparison,Category,Variant_Type,strelka_count,deepsomatic_count,mutect2_count,All_Three,Any_Two_or_More
0,RNA_realignment_normalized,Somatic,SNP,1148,280,2542,200,717
1,RNA_realignment_normalized,Somatic,INDEL,14,545,2876,3,520
2,RNA_realignment_normalized,All,SNP,17854,20426,2542,200,5290
3,RNA_realignment_normalized,All,INDEL,455,8755,2876,3,576



✓ Generated 4 Venn diagrams

RNA Realignment Focused Venn Diagrams:

RNA_realignment_normalized_Somatic_SNP:



RNA_realignment_normalized_Somatic_INDEL:



RNA_realignment_normalized_All_SNP:



RNA_realignment_normalized_All_INDEL:


In [27]:
# Consensus VCF 3-Way Comparison: DNA vs RNA Standard vs RNA Realignment
print("=" * 80)
print("CONSENSUS VCF 3-WAY COMPARISON")
print("=" * 80)

try:
    # 3-way comparison: DNA standard vs RNA standard vs RNA realignment
    # Generates per-category Venn diagrams for Somatic, Germline, Artifact, etc.
    consensus_summary, consensus_figures = caller_comparator.compare_consensus_vcfs(
        include_realignment=has_realignment
    )
    
    print("\n✓ Consensus VCF 3-Way Comparison Summary:")
    display(consensus_summary)
    
    print(f"\nGenerated {len(consensus_figures)} Venn diagrams by category and variant type")
    
    if has_realignment:
        print("\nNote: 3-way Venn diagrams compare DNA standard vs RNA standard vs RNA realignment")
    else:
        print("\nNote: 2-way Venn diagrams compare DNA standard vs RNA standard (no realignment available)")
    
    print("Each Venn shows overlap for specific FILTER categories (Somatic, Germline, etc.)")
    
    # Display Venn diagrams
    if consensus_figures:
        print("\nConsensus Per-Category Venn Diagrams:")
        for fig_name, fig in sorted(consensus_figures.items()):
            print(f"\n{fig_name}:")
            fig.show()
    
except Exception as e:
    print(f"✗ Error in consensus VCF comparison: {e}")
    import traceback
    traceback.print_exc()
    consensus_summary = None
    consensus_figures = {}

CONSENSUS VCF 3-WAY COMPARISON

3-Way Consensus Comparison: DNA vs RNA Standard vs RNA Realignment
  DNA: 2374372DT_vs_2374372DN.consensus.vcf.gz
  RNA Standard: 2374372RT_vs_2374372DN.consensus.vcf.gz
  RNA Realignment: 2374372RT_realign_vs_2374372DN.consensus.vcf.gz


  Generating per-category 3-way Venn diagrams...

✓ Consensus VCF 3-Way Comparison Summary:


,Comparison,Category,Variant_Type,DNA_Standard_count,RNA_Standard_count,RNA_Realignment_count,All_Three,Any_Two_or_More
0,Consensus 3way,Artifact,SNP,1762,7341,923,11,365
1,Consensus 3way,Artifact,INDEL,352,1445,1309,2,476
2,Consensus 3way,Germline,SNP,587,550,378,56,354
3,Consensus 3way,Germline,INDEL,11,5,4,0,3
4,Consensus 3way,NoConsensus,SNP,127047,172946,27113,8008,26293
5,Consensus 3way,NoConsensus,INDEL,3389,89800,8285,254,6256
6,Consensus 3way,Reference,SNP,4961,6012,4195,15,1250
7,Consensus 3way,Reference,INDEL,56,537,52,0,22
8,Consensus 3way,Somatic,SNP,148,2875,717,35,422
9,Consensus 3way,Somatic,INDEL,2,727,520,0,272



Generated 10 Venn diagrams by category and variant type

Note: 3-way Venn diagrams compare DNA standard vs RNA standard vs RNA realignment
Each Venn shows overlap for specific FILTER categories (Somatic, Germline, etc.)

Consensus Per-Category Venn Diagrams:

Consensus 3way_Artifact_INDEL:



Consensus 3way_Artifact_SNP:


...


Consensus 3way_NoConsensus_SNP:



Consensus 3way_Reference_INDEL:



Consensus 3way_Reference_SNP:



Consensus 3way_Somatic_INDEL:



Consensus 3way_Somatic_SNP:


## 8.5. Caller Comparison with Venn Diagrams

**Purpose**: Compare variant calling results across Strelka, DeepSomatic, and Mutect2 to identify high-confidence variants and caller-specific biases.

**Analysis Structure:**
1. **DNA Caller Comparison** (Standard): 4 Venn diagrams per category × variant type
2. **RNA Caller Comparison** (Standard): 4 Venn diagrams per category × variant type
3. **RNA Caller Comparison** (Realignment): 4 Venn diagrams per category × variant type
4. **Consensus 3-Way Comparison**: DNA vs RNA Standard vs RNA Realignment

**Key Features:**
- ✅ Stage-aware classification using `classify_by_filter()`
- ✅ Per-category Venn diagrams (Somatic, Germline, Artifact, etc.)
- ✅ Variant types: SNP, INDEL (OTHER excluded if all-zero)
- ✅ Interactive plotly visualizations with hover information

## 9. BAM Validation

**Purpose**: Validate variant calls by examining read support in BAM/CRAM alignment files.

**Validation Levels:**
1. **Standard BAM Validation**: DNA_TUMOR and RNA_TUMOR samples
2. **Comprehensive 4-Sample Validation**: DNA_TUMOR, DNA_NORMAL, RNA_TUMOR, RNA_TUMOR_realign (with realignment improvement metrics)

In [28]:
# Standard BAM validation
print("=" * 80)
print("STANDARD BAM VALIDATION")
print("=" * 80)

if standard_bams:
    validator = BAMValidator()
    
    # Find the best VCF for validation (prefer filtered_rescue > rescue > consensus)
    sample_vcf = None
    vcf_source = "unknown"
    
    rescue_files = discovery.get_rescue_files()
    if 'filtered_rescue' in standard_vcfs and standard_vcfs['filtered_rescue']:
        vcf_info = next(iter(standard_vcfs['filtered_rescue'].values()))
        # Extract path from metadata dict
        sample_vcf = vcf_info["path"] if isinstance(vcf_info, dict) else vcf_info
        vcf_source = "filtered_rescue"
    elif rescue_files:
        vcf_info = next(iter(rescue_files.values()))
        sample_vcf = vcf_info["path"] if isinstance(vcf_info, dict) else vcf_info
        vcf_source = "rescue"
    elif 'consensus' in standard_vcfs and standard_vcfs['consensus']:
        vcf_info = next(iter(standard_vcfs['consensus'].values()))
        sample_vcf = vcf_info["path"] if isinstance(vcf_info, dict) else vcf_info
        vcf_source = "consensus"
    
    if sample_vcf:
        print(f"\nValidating variants from: {sample_vcf.name}")
        print(f"VCF source: {vcf_source}")
        
        # Use tumor BAMs only
        tumor_bams = {k: v for k, v in standard_bams.items() if k in ["DNA_TUMOR", "RNA_TUMOR"]}
        
        if tumor_bams:
            print(f"Using {len(tumor_bams)} BAM file(s): {list(tumor_bams.keys())}")
            
            validation_results = validator.validate_variants(
                sample_vcf, 
                tumor_bams, 
                max_variants=50,
                stage_hint=vcf_source
            )
            
            validation_df = validator.summarize_validation(validation_results)
            
            if not validation_df.empty:
                print(f"\nValidation Summary:")
                print(f"Total variants validated: {len(validation_df)}")
                
                support_counts = validation_df["support"].value_counts()
                print(f"\nSupport distribution:")
                for support_type, count in support_counts.items():
                    pct = count / len(validation_df) * 100
                    print(f"  {support_type}: {count} ({pct:.1f}%)")
                
                display(validation_df.head(20))
        else:
            print("No tumor BAM files available.")
    else:
        print("No suitable VCF found for validation.")
else:
    print("No BAM files found.")


STANDARD BAM VALIDATION

Validating variants from: 2374372DT_vs_2374372DN_rescued_2374372RT_vs_2374372DN.filtered.vcf.gz
VCF source: filtered_rescue
Using 2 BAM file(s): ['DNA_TUMOR', 'RNA_TUMOR']



Validation Summary:
Total variants validated: 100

Support distribution:
  error: 91 (91.0%)
  unsupported: 9 (9.0%)


,chrom,pos,ref,alt,qual,filter,sample,bam_file,total_depth,ref_depth,alt_depth,alt_fraction,support,quality,mapping_quality,error
0,chr1,69151,T,G,None,NoConsensus,DNA_TUMOR,2374372DT.recal.cram,0,0,0,0.0,error,0.0,0.0,'pysam.libcalignedsegment.AlignedSegment' obje...
1,chr1,69151,T,G,None,NoConsensus,RNA_TUMOR,2374372RT.recal.cram,0,0,0,0.0,unsupported,0.0,0.0,NaN
2,chr1,69168,A,G,None,NoConsensus,DNA_TUMOR,2374372DT.recal.cram,0,0,0,0.0,error,0.0,0.0,'pysam.libcalignedsegment.AlignedSegment' obje...
3,chr1,69168,A,G,None,NoConsensus,RNA_TUMOR,2374372RT.recal.cram,0,0,0,0.0,unsupported,0.0,0.0,NaN
4,chr1,69849,G,A,None,Germline,DNA_TUMOR,2374372DT.recal.cram,0,0,0,0.0,error,0.0,0.0,'pysam.libcalignedsegment.AlignedSegment' obje...
5,chr1,69849,G,A,None,Germline,RNA_TUMOR,2374372RT.recal.cram,0,0,0,0.0,unsupported,0.0,0.0,NaN
6,chr1,69964,A,C,None,NoConsensus,DNA_TUMOR,2374372DT.recal.cram,0,0,0,0.0,error,0.0,0.0,'pysam.libcalignedsegment.AlignedSegment' obje...
7,chr1,69964,A,C,None,NoConsensus,RNA_TUMOR,2374372RT.recal.cram,0,0,0,0.0,unsupported,0.0,0.0,NaN
8,chr1,69999,A,G,None,NoConsensus,DNA_TUMOR,2374372DT.recal.cram,0,0,0,0.0,error,0.0,0.0,'pysam.libcalignedsegment.AlignedSegment' obje...
9,chr1,69999,A,G,None,NoConsensus,RNA_TUMOR,2374372RT.recal.cram,0,0,0,0.0,unsupported,0.0,0.0,NaN


In [29]:
# Comprehensive 4-sample BAM validation using RealignmentBAMValidator
if has_realignment and all_bam_files:
    print("=" * 80)
    print("COMPREHENSIVE 4-SAMPLE BAM VALIDATION")
    print("=" * 80)
    
    print(f"\nAvailable BAM files: {list(all_bam_files.keys())}")
    
    # Get filtered VCF from realignment workflow
    realignment_vcfs = all_workflow_vcfs.get('realignment', {})
    filtered_vcf = None
    
    if 'filtered_rescue' in realignment_vcfs and realignment_vcfs['filtered_rescue']:
        vcf_info = next(iter(realignment_vcfs['filtered_rescue'].values()))
        # Extract path from metadata dict
        filtered_vcf = vcf_info["path"] if isinstance(vcf_info, dict) else vcf_info
    elif 'rescue' in realignment_vcfs and realignment_vcfs['rescue']:
        vcf_info = next(iter(realignment_vcfs['rescue'].values()))
        filtered_vcf = vcf_info["path"] if isinstance(vcf_info, dict) else vcf_info
    
    if filtered_vcf:
        print(f"\nValidating: {filtered_vcf.name}")
        
        # Create comprehensive validator
        comprehensive_validator = RealignmentBAMValidator(
            filtered_vcf,
            all_bam_files
        )
        
        # Validate all samples
        comprehensive_validation = comprehensive_validator.validate_all_samples(max_variants=100)
        
        print(f"\nValidated {len(comprehensive_validation)} variants across 4 samples")
        display(comprehensive_validation.head(20))
        
        # Identify variants with improved realignment support
        if 'RNA_TUMOR_realign_VAF' in comprehensive_validation.columns:
            improved = comprehensive_validator.get_realignment_improvements(comprehensive_validation)
            print(f"\n✓ Found {len(improved)} variants with improved realignment support")
            
            if len(improved) > 0:
                print("\nTop 10 variants with improved support:")
                display(improved.head(10)[[
                    'CHROM', 'POS', 'REF', 'ALT',
                    'RNA_TUMOR_VAF', 'RNA_TUMOR_realign_VAF'
                ]])
        
        # Validation summary
        validation_summary = comprehensive_validator.summarize_validation(comprehensive_validation)
        print("\nValidation Summary:")
        for key, value in validation_summary.items():
            print(f"  {key}: {value}")
    else:
        print("No filtered VCF found for comprehensive validation.")
        comprehensive_validation = None
else:
    print("Skipping comprehensive validation (no realignment data or BAM files).")
    comprehensive_validation = None


COMPREHENSIVE 4-SAMPLE BAM VALIDATION

Available BAM files: ['DNA_TUMOR', 'RNA_TUMOR', 'DNA_NORMAL', 'RNA_TUMOR_realign']

Validating: 2374372DT_vs_2374372DN_rescued_2374372RT_realign_vs_2374372DN.filtered.vcf.gz

Validated 100 variants across 4 samples


,CHROM,POS,REF,ALT,QUAL,FILTER,DNA_TUMOR_ref_depth,DNA_TUMOR_alt_depth,DNA_TUMOR_total_depth,DNA_TUMOR_VAF,...,DNA_NORMAL_total_depth,DNA_NORMAL_VAF,DNA_NORMAL_support,RNA_TUMOR_realign_ref_depth,RNA_TUMOR_realign_alt_depth,RNA_TUMOR_realign_total_depth,RNA_TUMOR_realign_VAF,RNA_TUMOR_realign_support,RNA_VAF_improvement,Validation_status
0,chr1,69151,T,G,None,NoConsensus,5,1,12,0.083333,...,57,0.000000,unsupported,0,0,0,0.000000,unsupported,0.000000,weak_validation
1,chr1,69168,A,G,None,NoConsensus,4,1,11,0.090909,...,56,0.017857,minimal_support,0,0,0,0.000000,unsupported,0.000000,weak_validation
2,chr1,69849,G,A,None,Germline,10,4,16,0.250000,...,82,0.060976,supported,0,0,0,0.000000,unsupported,0.000000,validated_dna_only
3,chr1,69964,A,C,None,NoConsensus,14,0,14,0.000000,...,40,0.025000,minimal_support,0,0,0,0.000000,unsupported,0.000000,unsupported
4,chr1,69999,A,G,None,NoConsensus,11,1,12,0.083333,...,36,0.000000,unsupported,0,0,0,0.000000,unsupported,0.000000,weak_validation
5,chr1,924024,C,G,None,Germline,0,0,0,0.000000,...,0,0.000000,unsupported,0,6,6,1.000000,weak_support,0.000000,unsupported
6,chr1,924310,C,G,None,Germline,0,0,0,0.000000,...,0,0.000000,unsupported,0,18,18,1.000000,supported,0.000000,unsupported
7,chr1,924321,C,G,None,Germline,0,0,0,0.000000,...,0,0.000000,unsupported,0,17,17,1.000000,supported,0.000000,unsupported
8,chr1,930256,C,A,None,NoConsensus,48,17,84,0.202381,...,178,0.191011,supported,3,0,3,0.000000,unsupported,0.000000,validated_dna_only
9,chr1,942171,A,T,None,Reference,5,4,19,0.210526,...,19,0.000000,unsupported,43,0,43,0.000000,unsupported,0.000000,validated_dna_only



✓ Found 8 variants with improved realignment support

Top 10 variants with improved support:


,CHROM,POS,REF,ALT,RNA_TUMOR_VAF,RNA_TUMOR_realign_VAF
51,chr1,999613,TCTGGGGGCGGGGATAGGCGGGAGGTCCAGGTCAGCTGCGACCCAG...,T,0.494881,0.795082
52,chr1,999787,ACTGCGGGTCGGGCACCGGCTGAGTCCCGCGTCCCTCCCGCCCCCC...,A,0.362500,0.585859
75,chr1,1046086,AAGGTGAGGGGTGTGGGATGTGAAGGGGAGTGGGGAGGAGGCCTCG...,A,0.835443,0.969697
96,chr1,1051368,TGGTATGTGGGGGCGGGGCGTCCCAGCAGGGCCTCCGGGGCGGGCG...,T,0.870091,0.994505
12,chr1,942487,CGGTAGGTGCGGGGAGGCGGGCGGGGCCGCGCGGCCCGGGAGGCGG...,C,0.834646,0.924242
44,chr1,973862,A,G,0.562500,0.636364
79,chr1,1046502,T,A,0.100437,0.169643
16,chr1,943805,CCAGGTGAGACGCTGGGGAGTGAGGTCAGGGTCTCCAGACCACAGC...,C,0.807229,0.860000



Validation Summary:
  total_variants: 100
  validation_status_counts: {'validated_dna_only': 49, 'validated': 31, 'weak_validation': 9, 'unsupported': 7, 'validated_with_realignment_improvement': 4}
  realignment_improvements: 9
  sample_statistics: {'DNA_TUMOR': {'mean_VAF': np.float64(0.16781902043160232), 'median_VAF': np.float64(0.09464875058934465), 'mean_depth': np.float64(328.19), 'median_depth': np.float64(309.0), 'supported_count': 84, 'weak_support_count': 3, 'unsupported_count': 7}, 'DNA_NORMAL': {'mean_VAF': np.float64(0.15714339621636783), 'median_VAF': np.float64(0.09935789117945251), 'mean_depth': np.float64(457.59), 'median_depth': np.float64(419.0), 'supported_count': 81, 'weak_support_count': 5, 'unsupported_count': 10}, 'RNA_TUMOR': {'mean_VAF': np.float64(0.20238975020599032), 'median_VAF': np.float64(0.07484202211690363), 'mean_depth': np.float64(173.73), 'median_depth': np.float64(185.0), 'supported_count': 66, 'weak_support_count': 5, 'unsupported_count': 29}, '

## 10. Rescue Variant Tiering & IGV-Reports Visualization

### CxDy Hybrid Tiering System

The tiering system uses a **CxDy format** combining caller support and database evidence:

**Caller Tiers (C1-C7):**
- **C1**: ≥2 DNA callers + ≥2 RNA callers (highest confidence)
- **C2**: ≥2 DNA callers + 0-1 RNA callers
- **C3**: 0-1 DNA callers + ≥2 RNA callers
- **C4**: 1 DNA caller + 1 RNA caller
- **C5**: 1 DNA caller only
- **C6**: 1 RNA caller only
- **C7**: 0 DNA + 0 RNA callers (lowest confidence)

**Database Tiers (D0-D1):**
- **D1**: Has database support (gnomAD AF > 0, COSMIC count > 0, REDIportal, or DARNED)
- **D0**: No database support

**Total: 14 tiers** (C1D1, C1D0, C2D1, C2D0, ... C7D1, C7D0)

In [30]:
# Rescue variant tiering with CxDy system
print("=" * 80)
print("RESCUE VARIANT TIERING (CxDy Hybrid System)")
print("=" * 80)
print("CxDy format: Caller tier (C1-C7) × Database tier (D0-D1)")
print("  C1-C7: Based on DNA/RNA caller support counts")
print("  D0-D1: Based on database evidence (gnomAD, COSMIC, REDIportal, DARNED)")
print()

import subprocess

# Get rescue files (deduplicated)
rescue_files = discovery.get_rescue_files()
seen_paths = set()
unique_rescue_files = {}
for key, vcf_info in rescue_files.items():
    # Extract path from metadata dict
    path = vcf_info["path"] if isinstance(vcf_info, dict) else vcf_info
    path_str = str(path.resolve())
    if path_str not in seen_paths:
        unique_rescue_files[key] = path
        seen_paths.add(path_str)

rescue_files = unique_rescue_files
print(f"Found {len(rescue_files)} unique rescue VCF(s)")

# Output directory
igvreports_dir = OUTPUT_DIR / "igv_reports"
igvreports_dir.mkdir(exist_ok=True)

K = 3  # Representatives per tier

for rescue_name, rescue_path in rescue_files.items():
    print(f"\nProcessing: {rescue_name}")
    print("-" * 40)
    
    # Compute tiers using CxDy system
    tiered = tier_rescue_variants(rescue_path)
    if tiered.empty:
        print(f"  No variants parsed")
        continue
    
    print(f"  Total variants: {len(tiered):,}")
    
    # Show tier distribution by category using CxDy tiers
    categories = sorted(tiered["filter_category"].unique())
    for cat in categories:
        cat_data = tiered[tiered["filter_category"] == cat]
        tier_counts = cat_data["tier"].value_counts()
        pct_total = (len(cat_data) / len(tiered)) * 100
        print(f"\n  {cat} ({len(cat_data):,} variants, {pct_total:.1f}%):")
        
        # Display tiers in CxDy order
        for tier in TIER_ORDER:
            if tier in tier_counts.index:
                count = tier_counts[tier]
                pct = (count / len(cat_data)) * 100
                print(f"    {tier}: {count:,} ({pct:.1f}%)")

RESCUE VARIANT TIERING (CxDy Hybrid System)
CxDy format: Caller tier (C1-C7) × Database tier (D0-D1)
  C1-C7: Based on DNA/RNA caller support counts
  D0-D1: Based on database evidence (gnomAD, COSMIC, REDIportal, DARNED)

Found 0 unique rescue VCF(s)


In [31]:
# Generate IGV-reports (if available)
if rescue_files and check_igv_reports_available():
    print("\n" + "=" * 80)
    print("GENERATING IGV-REPORTS")
    print("=" * 80)
    
    # Prepare BAM files for visualization
    vis_bams = {k: v for k, v in standard_bams.items() 
                if k in ["DNA_TUMOR", "DNA_NORMAL", "RNA_TUMOR"]}
    
    if not REF_FASTA.exists():
        print(f"⚠ Reference FASTA not found: {REF_FASTA}")
    else:
        for rescue_name, rescue_path in rescue_files.items():
            print(f"\nProcessing: {rescue_name}")
            
            tiered = tier_rescue_variants(rescue_path)
            if tiered.empty:
                continue
            
            tier_output_dir = igvreports_dir / rescue_name
            
            try:
                tier_reports = organize_by_category_tier(
                    tiered, rescue_path, vis_bams, REF_FASTA, tier_output_dir, k_per_tier=K
                )
                print(f"  ✓ Generated reports for {len(tier_reports)} category-tier combinations")
                print(f"  📊 Reports directory: {tier_output_dir}")
            except Exception as e:
                print(f"  ✗ Error: {e}")
else:
    if not rescue_files:
        print("No rescue files for IGV-reports.")
    else:
        print("igv-reports not installed. Install with: pip install igv-reports")

No rescue files for IGV-reports.


In [32]:
# Generate tier visualization reports
print("\n" + "=" * 80)
print("TIER VISUALIZATION REPORTS")
print("=" * 80)

tier_viz_dir = OUTPUT_DIR / "tier_visualizations"
tier_viz_dir.mkdir(exist_ok=True)

for rescue_name, rescue_path in rescue_files.items():
    print(f"\nProcessing: {rescue_name}")
    
    tiered_df = tier_rescue_variants(rescue_path)
    if not tiered_df.empty:
        print(f"  ✓ Loaded {len(tiered_df):,} variants")
        
        rescue_viz_dir = tier_viz_dir / rescue_name
        rescue_viz_dir.mkdir(parents=True, exist_ok=True)
        
        report = create_tier_visualization_report(tiered_df, output_dir=rescue_viz_dir)
        print(f"  ✓ Generated visualizations")
        print(f"  📊 Saved to: {rescue_viz_dir}")


TIER VISUALIZATION REPORTS


## 11. Export Results

In [33]:
# Export all results
print("=" * 80)
print("EXPORTING RESULTS")
print("=" * 80)

# Export standard workflow results
try:
    if hasattr(standard_aggregator, 'export_report'):
        standard_aggregator.export_report(OUTPUT_DIR, format='both')
        print(f"✓ Standard workflow statistics exported")
except Exception as e:
    print(f"✗ Error exporting standard statistics: {e}")

# Export realignment comparison results
if has_realignment and comparator:
    realignment_output_dir = OUTPUT_DIR / 'realignment_analysis'
    realignment_output_dir.mkdir(exist_ok=True)
    
    try:
        # Export comparison tables
        if 'rna_count_comparison' in dir():
            rna_count_comparison.to_csv(
                realignment_output_dir / 'rna_variant_count_comparison.csv', index=False
            )
        if 'rna_category_comparison' in dir():
            rna_category_comparison.to_csv(
                realignment_output_dir / 'rna_category_distribution_comparison.csv', index=False
            )
        if 'integrative_view' in dir():
            integrative_view.to_csv(
                realignment_output_dir / 'integrative_view.csv', index=False
            )
        
        # Export comparison report
        comparator.export_comparison_report(realignment_output_dir)
        print(f"✓ Realignment comparison results exported")
    except Exception as e:
        print(f"✗ Error exporting realignment results: {e}")
    
    # Export comprehensive validation
    if comprehensive_validation is not None:
        try:
            comprehensive_validation.to_csv(
                realignment_output_dir / 'comprehensive_bam_validation.csv', index=False
            )
            print(f"✓ Comprehensive BAM validation exported")
        except Exception as e:
            print(f"✗ Error exporting validation: {e}")

# List all exported files
print(f"\nExported files in {OUTPUT_DIR}:")
for file_path in sorted(OUTPUT_DIR.rglob("*")):
    if file_path.is_file():
        print(f"  - {file_path.relative_to(OUTPUT_DIR)}")

EXPORTING RESULTS
✓ Report exported to Excel: P2374372_statistics_output/vcf_statistics_report_standard.xlsx
✓ Report exported to CSV files in: P2374372_statistics_output/csv_reports_standard
✓ Standard workflow statistics exported
✓ Workflow comparison report exported to Excel: P2374372_statistics_output/realignment_analysis/workflow_comparison_report.xlsx
✓ Workflow comparison CSV files exported to: P2374372_statistics_output/realignment_analysis/comparison_csv
✓ Realignment comparison results exported
✓ Comprehensive BAM validation exported

Exported files in P2374372_statistics_output:
  - csv_reports_standard/category_distribution.csv
  - csv_reports_standard/stage_progression.csv
  - csv_reports_standard/variant_count_summary.csv
  - realignment_analysis/comparison_csv/integrative_view.csv
  - realignment_analysis/comparison_csv/realignment_impact.csv
  - realignment_analysis/comparison_csv/rna_annotation_stages.csv
  - realignment_analysis/comparison_csv/rna_category_distributio

## 12. Analysis Summary

In [34]:
print("=" * 80)
print("VCF STATISTICS ANALYSIS SUMMARY")
print("=" * 80)

print(f"\n✅ Analysis Complete!")
print(f"\n📁 Configuration:")
print(f"   Sample ID: {SAMPLE_ID}")
print(f"   Base directory: {BASE_DIR}")
print(f"   Output directory: {OUTPUT_DIR}")

print(f"\n🔄 Detected Workflows:")
for wf in detected_workflows:
    status = "✅ Processed" if wf == WorkflowType.STANDARD or (wf == WorkflowType.REALIGNMENT and has_realignment) else "⚠️  Skipped"
    print(f"   {status} {wf.value.capitalize()}")

print(f"\n📊 Statistics Processed:")
print(f"   Standard workflow: {len(standard_stats)} stages, {sum(len(files) for files in standard_stats.values())} VCF files")
if has_realignment:
    print(f"   Realignment workflow: {len(realignment_stats)} stages, {sum(len(files) for files in realignment_stats.values())} VCF files")

print(f"\n🏷️  Variant Categories:")
for cat in CATEGORY_ORDER:
    color = CATEGORY_COLORS.get(cat, "#8A8A8A")
    print(f"   {cat} ({color})")

print(f"\n📊 CxDy Tiering System:")
print(f"   Total tiers: {len(TIER_ORDER)}")
print(f"   Caller tiers: C1-C7 (DNA/RNA caller support)")
print(f"   Database tiers: D0-D1 (gnomAD, COSMIC, REDIportal, DARNED)")

print(f"\n✓ Output Locations:")
print(f"   Statistics & summaries: {OUTPUT_DIR}/")
print(f"   Tier visualizations: {OUTPUT_DIR}/tier_visualizations/")
if check_igv_reports_available():
    print(f"   IGV reports: {OUTPUT_DIR}/igv_reports/")
if has_realignment:
    print(f"   Realignment analysis: {OUTPUT_DIR}/realignment_analysis/")

print(f"\n📈 Key Metrics:")
if 'variant_summary' in dir() and not variant_summary.empty:
    total_variants = variant_summary['Total'].sum() if 'Total' in variant_summary.columns else 0
    print(f"   Total variants processed: {total_variants:,}")
if has_realignment and 'realignment_impact' in dir():
    improvement = realignment_impact.get('realignment_improvement', 0)
    print(f"   Realignment improvement: {improvement:.2f}%")

print("\n" + "=" * 80)
print("✅ All analyses completed successfully")
print("=" * 80)

VCF STATISTICS ANALYSIS SUMMARY

✅ Analysis Complete!

📁 Configuration:
   Sample ID: P2374372
   Base directory: /t9k/mnt/hdd/work/Vax/sequencing/aim_exp/aim_11/output/P2374372
   Output directory: P2374372_statistics_output

🔄 Detected Workflows:
   ✅ Processed Standard
   ✅ Processed Realignment

📊 Statistics Processed:
   Standard workflow: 6 stages, 12 VCF files
   Realignment workflow: 6 stages, 8 VCF files

🏷️  Variant Categories:
   Somatic (#636EFA)
   Germline (#00CC96)
   Reference (#FFA15A)
   Artifact (#EF553B)
   RNAedit (#AB63FA)
   NoConsensus (#8A8A8A)

📊 CxDy Tiering System:
   Total tiers: 14
   Caller tiers: C1-C7 (DNA/RNA caller support)
   Database tiers: D0-D1 (gnomAD, COSMIC, REDIportal, DARNED)

✓ Output Locations:
   Statistics & summaries: P2374372_statistics_output/
   Tier visualizations: P2374372_statistics_output/tier_visualizations/
   IGV reports: P2374372_statistics_output/igv_reports/
   Realignment analysis: P2374372_statistics_output/realignment_ana

---

## Implementation Notes & Module Reference

**Core Modules** (`bin/vcf_stats/`):
- **workflow.py**: WorkflowManager, WorkflowType, WorkflowConfig (auto-detects standard/realignment)
- **file_discovery.py**: VCFFileDiscovery (workflow-aware, stage-aware file discovery)
- **vcf_processor.py**: VCFStatisticsExtractor (stage-aware classification with caller-specific logic)
- **statistics_aggregator.py**: StatisticsAggregator (summary tables, category aggregation)
- **visualizer.py**: VCFVisualizer (pipeline plots, tier distributions, workflow comparisons)
- **comparison.py**: WorkflowComparator (standard vs realignment metrics)
- **caller_comparison.py**: CallerComparator (3-way Venn diagrams, per-category comparisons)
- **bam_validator.py**: BAMValidator, RealignmentBAMValidator (variant read support validation)
- **tiering.py**: tier_rescue_variants, TieringEngine (CxDy hybrid tiering system)
- **tiering_viz.py**: TierVisualizer, create_tier_visualization_report (tier plots)
- **igv_reports.py**: organize_by_category_tier (IGV-reports generation)

**Shared Configuration** (`bin/common/`):
- **vcf_config.py**: CATEGORY_ORDER, CATEGORY_COLORS, VCF_STAGE_ORDER, get_variant_type()
- **tier_config.py**: TIER_ORDER, TIER_COLORS, get_tier_display_name(), get_tier_quality()

**Key Implementation Details:**
1. **CxDy Tiering System**: Combines caller support (C1-C7) with database evidence (D0-D1) for 14-tier classification
2. **Stage-Aware Classification**: Uses different logic for normalized (caller-specific) vs. filtered (FILTER-based) stages
3. **Workflow Auto-Detection**: WorkflowManager automatically detects standard/realignment from directory structure
4. **Filtered Rescue Priority**: Always uses `filtered_rescue` VCF for tiering (has COSMIC/gnomAD/RNAedit annotations)
5. **Caller-Specific Logic**: Strelka uses FORMAT/DP, DeepSomatic uses INFO fields, Mutect2 uses FORMAT/AF thresholds

**For detailed implementation documentation, see:**
- Development docs: `dev_docs/` directory
- Module docstrings: Each Python file in `bin/vcf_stats/`
- Test suite: `tests/` directory